## How Weekly Fantasy Football Works:

In [ ]:
% matplotlib inline
from skimage.io import imread
from skimage.transform import resize 
from matplotlib import pyplot as plt
import matplotlib.cm as cm

ffdiagram_url=("http://www.dailyfantasysports101.com/wp-content/uploads/2014/01/draftkings-football-howtoplay-545.gif")
ffdiagram=imread(ffdiagram_url)
ypixels, xpixels, bands = ffdiagram.shape
dpi=72
xinch=xpixels/dpi
yinch=ypixels/dpi
fig=plt.figure(figsize=(xinch,yinch))
ax = plt.axes([0, 0, 1, 1], frameon=False, xticks=[],yticks=[])
ax.imshow(ffdiagram, interpolation='none')
plt.imshow(ffdiagram)
plt.axis('off')
plt.show()

# Data Gathering and Cleaning

## Import Libraries:

In [ ]:
%matplotlib inline 
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pattern import web
import re
import csv
import numpy as np

## Define Functions:

In [ ]:
def get_table(html):
    dom = web.Element(html)
    tbl = dom.by_class("mod-table align-center nowrap")
    return tbl
  
def table_type(tbl):
    return tbl('th')[0].content

def get_stats_rb(tables):
    result = defaultdict(dict)
    for tbl in tables:
        headers = tbl('tr')
        first_header = headers[1]
        th_s = first_header('th')
        stats_header = [val.content.replace('&pos;=rb&week;"&gt;','') for val in th_s]
        stats_header.remove("PLAYERPLAYER")
        stats_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        rows = tbl('tr')[2:]
        for i in range(0, len(rows) - 1):
            row = rows[i]
            tds = row('td')
            player_name = tds[0].content.replace('"&gt;','')
            player_name=re.sub('^[0-9 ]*', '', player_name)
            stats = [tds[colindex].content for colindex in stats_indices]
            subdict = dict(zip(stats_header, stats))
            result[player_name].update(subdict)
    return result

def get_stats_qb(tables):
    result = defaultdict(dict)
    for tbl in tables:
        headers = tbl('tr')
        first_header = headers[1]
        th_s = first_header('th')
        stats_header = [val.content.replace('&pos;=qb&week;"&gt;','') for val in th_s]
        stats_header.remove("PLAYERPLAYER")
        stats_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        rows = tbl('tr')[2:]
        for i in range(0, len(rows) - 1):
            row = rows[i]
            tds = row('td')
            player_name = tds[0].content.replace('"&gt;','')
            player_name=re.sub('^[0-9 ]*', '', player_name)
            stats = [tds[colindex].content for colindex in stats_indices]
            subdict = dict(zip(stats_header, stats))
            result[player_name].update(subdict)
    return result

def get_stats_wr(tables):
    result = defaultdict(dict)
    for tbl in tables:
        headers = tbl('tr')
        first_header = headers[1]
        th_s = first_header('th')
        stats_header = [val.content.replace('&pos;=wr&week;"&gt;','') for val in th_s]
        stats_header.remove("PLAYERPLAYER")
        stats_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        rows = tbl('tr')[2:]
        for i in range(0, len(rows) - 1):
            row = rows[i]
            tds = row('td')
            player_name = tds[0].content.replace('"&gt;','')
            player_name=re.sub('^[0-9 ]*', '', player_name)
            stats = [tds[colindex].content for colindex in stats_indices]
            subdict = dict(zip(stats_header, stats))
            result[player_name].update(subdict)
    return result

def get_stats_te(tables):
    result = defaultdict(dict)
    for tbl in tables:
        headers = tbl('tr')
        first_header = headers[1]
        th_s = first_header('th')
        stats_header = [val.content.replace('&pos;=te&week;"&gt;','') for val in th_s]
        stats_header.remove("PLAYERPLAYER")
        stats_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
        rows = tbl('tr')[2:]
        for i in range(0, len(rows) - 1):
            row = rows[i]
            tds = row('td')
            player_name = tds[0].content.replace('"&gt;','')
            player_name=re.sub('^[0-9 ]*', '', player_name)
            stats = [tds[colindex].content for colindex in stats_indices]
            subdict = dict(zip(stats_header, stats))
            result[player_name].update(subdict)
    return result

def get_stats_df(tables):
    result = defaultdict(dict)
    for tbl in tables:
        headers = tbl('tr')
        first_header = headers[1]
        th_s = first_header('th')
        stats_header = [val.content.replace('&pos;=df&week;"&gt;','') for val in th_s]
        stats_header.remove("PLAYERPLAYER")
        stats_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        rows = tbl('tr')[2:]
        for i in range(0, len(rows) - 1):
            row = rows[i]
            tds = row('td')
            player_name = tds[0].content.replace('roster/"&gt;','')
            player_name=re.sub('(/nfl/teams/)', '', player_name)
            stats = [tds[colindex].content for colindex in stats_indices]
            subdict = dict(zip(stats_header, stats))
            result[player_name].update(subdict)
    return result

def make_df(result):
    df = pd.DataFrame.from_dict(result, orient='index')
    df.columns=['PLAYS','RYD','NFL','PTD','RUN','PASS','CMP','INT','FUM','PYDS','FPTS','RTD']
    df[['PLAYS','RYD','PTD','RUN','PASS','CMP','INT','FUM','PYDS','FPTS','RTD']] = df[['PLAYS','RYD','PTD','RUN','PASS','CMP','INT','FUM','PYDS','FPTS','RTD']].astype(int)
    return df

def make_dfdf(result):
    df = pd.DataFrame.from_dict(result, orient='index')
    df.columns=['RYDA','SACK','FR','NFL','TYDA','SFTY','INT','TD','PYDA','FPTS']
    df[['RYDA','SACK','FR','TYDA','SFTY','INT','TD','PYDA','FPTS']] = df[['RYDA','SACK','FR','TYDA','SFTY','INT','TD','PYDA','FPTS']].astype(int)
    return df

## Collect and Clean Data:

### Regular Expression Variables:

In [ ]:
header=re.compile(ur'(<th colspan)')
cls=re.compile(ur'(class="t_std_bold_red_cell sorted")')
cls1=re.compile(ur'(class="t_std_left align-left")')
cls2=re.compile(ur'(class="")')
ab1=re.compile(ur'(<a href=")')
ab2=re.compile(ur'\n')
ab3=re.compile(ur'\r')
ab4=re.compile(ur'[?]')
ab5=re.compile(ur'/nfl/player/')
ab6=re.compile(ur'  ')
ab7=re.compile(ur'(col=)')
ab8_1=re.compile(ur'(=1)')
ab8_2=re.compile(ur'(=2)')
ab8_3=re.compile(ur'(=3)')
ab8_4=re.compile(ur'(=4)')
ab8_5=re.compile(ur'(=5)')
ab8_6=re.compile(ur'(=6)')
ab8_7=re.compile(ur'(=7)')
ab8_8=re.compile(ur'(=8)')
ab8_9=re.compile(ur'(=9)')
ab8_10=re.compile(ur'(=10)')
ab8_11=re.compile(ur'(=11)')

### Running Backs

In [ ]:
#Week 1
rb_url_1 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=1&pos=rb&col=FPTS&ccs=96662'
rb_html_1 = requests.get(rb_url_1).text

rb_html_1=re.sub(header,'',rb_html_1)
rb_html_1=re.sub(cls,'',rb_html_1)
rb_html_1=re.sub(cls1,'',rb_html_1)
rb_html_1=re.sub(cls2,'',rb_html_1)
rb_html_1=re.sub(ab1,'',rb_html_1)
rb_html_1=re.sub(ab2,'',rb_html_1)
rb_html_1=re.sub(ab3,'',rb_html_1)
rb_html_1=re.sub(ab4,'',rb_html_1)
rb_html_1=re.sub(ab5,'',rb_html_1)
rb_html_1=re.sub(ab6,'',rb_html_1)
rb_html_1=re.sub(ab7,'',rb_html_1)
rb_html_1=re.sub(ab8_1,'',rb_html_1)

#Week 2
rb_url_2 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=2&pos=rb&col=FPTS&ccs=96662'
rb_html_2 = requests.get(rb_url_2).text

rb_html_2=re.sub(header,'',rb_html_2)
rb_html_2=re.sub(cls,'',rb_html_2)
rb_html_2=re.sub(cls1,'',rb_html_2)
rb_html_2=re.sub(cls2,'',rb_html_2)
rb_html_2=re.sub(ab1,'',rb_html_2)
rb_html_2=re.sub(ab2,'',rb_html_2)
rb_html_2=re.sub(ab3,'',rb_html_2)
rb_html_2=re.sub(ab4,'',rb_html_2)
rb_html_2=re.sub(ab5,'',rb_html_2)
rb_html_2=re.sub(ab6,'',rb_html_2)
rb_html_2=re.sub(ab7,'',rb_html_2)
rb_html_2=re.sub(ab8_2,'',rb_html_2)

#Week 3
rb_url_3 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=3&pos=rb&col=FPTS&ccs=96662'
rb_html_3 = requests.get(rb_url_3).text

rb_html_3=re.sub(header,'',rb_html_3)
rb_html_3=re.sub(cls,'',rb_html_3)
rb_html_3=re.sub(cls1,'',rb_html_3)
rb_html_3=re.sub(cls2,'',rb_html_3)
rb_html_3=re.sub(ab1,'',rb_html_3)
rb_html_3=re.sub(ab2,'',rb_html_3)
rb_html_3=re.sub(ab3,'',rb_html_3)
rb_html_3=re.sub(ab4,'',rb_html_3)
rb_html_3=re.sub(ab5,'',rb_html_3)
rb_html_3=re.sub(ab6,'',rb_html_3)
rb_html_3=re.sub(ab7,'',rb_html_3)
rb_html_3=re.sub(ab8_3,'',rb_html_3)

#Week 4
rb_url_4 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=4&pos=rb&col=FPTS&ccs=96662'
rb_html_4 = requests.get(rb_url_4).text

rb_html_4=re.sub(header,'',rb_html_4)
rb_html_4=re.sub(cls,'',rb_html_4)
rb_html_4=re.sub(cls1,'',rb_html_4)
rb_html_4=re.sub(cls2,'',rb_html_4)
rb_html_4=re.sub(ab1,'',rb_html_4)
rb_html_4=re.sub(ab2,'',rb_html_4)
rb_html_4=re.sub(ab3,'',rb_html_4)
rb_html_4=re.sub(ab4,'',rb_html_4)
rb_html_4=re.sub(ab5,'',rb_html_4)
rb_html_4=re.sub(ab6,'',rb_html_4)
rb_html_4=re.sub(ab7,'',rb_html_4)
rb_html_4=re.sub(ab8_4,'',rb_html_4)

#Week 5
rb_url_5 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=5&pos=rb&col=FPTS&ccs=96662'
rb_html_5 = requests.get(rb_url_5).text

rb_html_5=re.sub(header,'',rb_html_5)
rb_html_5=re.sub(cls,'',rb_html_5)
rb_html_5=re.sub(cls1,'',rb_html_5)
rb_html_5=re.sub(cls2,'',rb_html_5)
rb_html_5=re.sub(ab1,'',rb_html_5)
rb_html_5=re.sub(ab2,'',rb_html_5)
rb_html_5=re.sub(ab3,'',rb_html_5)
rb_html_5=re.sub(ab4,'',rb_html_5)
rb_html_5=re.sub(ab5,'',rb_html_5)
rb_html_5=re.sub(ab6,'',rb_html_5)
rb_html_5=re.sub(ab7,'',rb_html_5)
rb_html_5=re.sub(ab8_5,'',rb_html_5)

#Week 6
rb_url_6 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=6&pos=rb&col=FPTS&ccs=96662'
rb_html_6 = requests.get(rb_url_6).text

rb_html_6=re.sub(header,'',rb_html_6)
rb_html_6=re.sub(cls,'',rb_html_6)
rb_html_6=re.sub(cls1,'',rb_html_6)
rb_html_6=re.sub(cls2,'',rb_html_6)
rb_html_6=re.sub(ab1,'',rb_html_6)
rb_html_6=re.sub(ab2,'',rb_html_6)
rb_html_6=re.sub(ab3,'',rb_html_6)
rb_html_6=re.sub(ab4,'',rb_html_6)
rb_html_6=re.sub(ab5,'',rb_html_6)
rb_html_6=re.sub(ab6,'',rb_html_6)
rb_html_6=re.sub(ab7,'',rb_html_6)
rb_html_6=re.sub(ab8_6,'',rb_html_6)

#Week 7
rb_url_7 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=7&pos=rb&col=FPTS&ccs=96662'
rb_html_7 = requests.get(rb_url_7).text

rb_html_7=re.sub(header,'',rb_html_7)
rb_html_7=re.sub(cls,'',rb_html_7)
rb_html_7=re.sub(cls1,'',rb_html_7)
rb_html_7=re.sub(cls2,'',rb_html_7)
rb_html_7=re.sub(ab1,'',rb_html_7)
rb_html_7=re.sub(ab2,'',rb_html_7)
rb_html_7=re.sub(ab3,'',rb_html_7)
rb_html_7=re.sub(ab4,'',rb_html_7)
rb_html_7=re.sub(ab5,'',rb_html_7)
rb_html_7=re.sub(ab6,'',rb_html_7)
rb_html_7=re.sub(ab7,'',rb_html_7)
rb_html_7=re.sub(ab8_7,'',rb_html_7)

#Week 8
rb_url_8 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=8&pos=rb&col=FPTS&ccs=96662'
rb_html_8 = requests.get(rb_url_8).text

rb_html_8=re.sub(header,'',rb_html_8)
rb_html_8=re.sub(cls,'',rb_html_8)
rb_html_8=re.sub(cls1,'',rb_html_8)
rb_html_8=re.sub(cls2,'',rb_html_8)
rb_html_8=re.sub(ab1,'',rb_html_8)
rb_html_8=re.sub(ab2,'',rb_html_8)
rb_html_8=re.sub(ab3,'',rb_html_8)
rb_html_8=re.sub(ab4,'',rb_html_8)
rb_html_8=re.sub(ab5,'',rb_html_8)
rb_html_8=re.sub(ab6,'',rb_html_8)
rb_html_8=re.sub(ab7,'',rb_html_8)
rb_html_8=re.sub(ab8_8,'',rb_html_8)

#Week 9
rb_url_9 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=9&pos=rb&col=FPTS&ccs=96662'
rb_html_9 = requests.get(rb_url_9).text

rb_html_9=re.sub(header,'',rb_html_9)
rb_html_9=re.sub(cls,'',rb_html_9)
rb_html_9=re.sub(cls1,'',rb_html_9)
rb_html_9=re.sub(cls2,'',rb_html_9)
rb_html_9=re.sub(ab1,'',rb_html_9)
rb_html_9=re.sub(ab2,'',rb_html_9)
rb_html_9=re.sub(ab3,'',rb_html_9)
rb_html_9=re.sub(ab4,'',rb_html_9)
rb_html_9=re.sub(ab5,'',rb_html_9)
rb_html_9=re.sub(ab6,'',rb_html_9)
rb_html_9=re.sub(ab7,'',rb_html_9)
rb_html_9=re.sub(ab8_9,'',rb_html_9)

#Week 10
rb_url_10 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=10&pos=rb&col=FPTS&ccs=96662'
rb_html_10 = requests.get(rb_url_10).text

rb_html_10=re.sub(header,'',rb_html_10)
rb_html_10=re.sub(cls,'',rb_html_10)
rb_html_10=re.sub(cls1,'',rb_html_10)
rb_html_10=re.sub(cls2,'',rb_html_10)
rb_html_10=re.sub(ab1,'',rb_html_10)
rb_html_10=re.sub(ab2,'',rb_html_10)
rb_html_10=re.sub(ab3,'',rb_html_10)
rb_html_10=re.sub(ab4,'',rb_html_10)
rb_html_10=re.sub(ab5,'',rb_html_10)
rb_html_10=re.sub(ab6,'',rb_html_10)
rb_html_10=re.sub(ab7,'',rb_html_10)
rb_html_10=re.sub(ab8_10,'',rb_html_10)

#Week 11
rb_url_11 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=11&pos=rb&col=FPTS&ccs=96662'
rb_html_11 = requests.get(rb_url_11).text

rb_html_11=re.sub(header,'',rb_html_11)
rb_html_11=re.sub(cls,'',rb_html_11)
rb_html_11=re.sub(cls1,'',rb_html_11)
rb_html_11=re.sub(cls2,'',rb_html_11)
rb_html_11=re.sub(ab1,'',rb_html_11)
rb_html_11=re.sub(ab2,'',rb_html_11)
rb_html_11=re.sub(ab3,'',rb_html_11)
rb_html_11=re.sub(ab4,'',rb_html_11)
rb_html_11=re.sub(ab5,'',rb_html_11)
rb_html_11=re.sub(ab6,'',rb_html_11)
rb_html_11=re.sub(ab7,'',rb_html_11)
rb_html_11=re.sub(ab8_11,'',rb_html_11)

### Quarterbacks

In [ ]:
#Week 1
qb_url_1 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=1&pos=qb&col=FPTS&ccs=96662'
qb_html_1 = requests.get(qb_url_1).text

qb_html_1=re.sub(header,'',qb_html_1)
qb_html_1=re.sub(cls,'',qb_html_1)
qb_html_1=re.sub(cls1,'',qb_html_1)
qb_html_1=re.sub(cls2,'',qb_html_1)
qb_html_1=re.sub(ab1,'',qb_html_1)
qb_html_1=re.sub(ab2,'',qb_html_1)
qb_html_1=re.sub(ab3,'',qb_html_1)
qb_html_1=re.sub(ab4,'',qb_html_1)
qb_html_1=re.sub(ab5,'',qb_html_1)
qb_html_1=re.sub(ab6,'',qb_html_1)
qb_html_1=re.sub(ab7,'',qb_html_1)
qb_html_1=re.sub(ab8_1,'',qb_html_1)

#Week 2
qb_url_2 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=2&pos=qb&col=FPTS&ccs=96662'
qb_html_2 = requests.get(qb_url_2).text

qb_html_2=re.sub(header,'',qb_html_2)
qb_html_2=re.sub(cls,'',qb_html_2)
qb_html_2=re.sub(cls1,'',qb_html_2)
qb_html_2=re.sub(cls2,'',qb_html_2)
qb_html_2=re.sub(ab1,'',qb_html_2)
qb_html_2=re.sub(ab2,'',qb_html_2)
qb_html_2=re.sub(ab3,'',qb_html_2)
qb_html_2=re.sub(ab4,'',qb_html_2)
qb_html_2=re.sub(ab5,'',qb_html_2)
qb_html_2=re.sub(ab6,'',qb_html_2)
qb_html_2=re.sub(ab7,'',qb_html_2)
qb_html_2=re.sub(ab8_2,'',qb_html_2)

#Week 3
qb_url_3 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=3&pos=qb&col=FPTS&ccs=96662'
qb_html_3 = requests.get(qb_url_3).text

qb_html_3=re.sub(header,'',qb_html_3)
qb_html_3=re.sub(cls,'',qb_html_3)
qb_html_3=re.sub(cls1,'',qb_html_3)
qb_html_3=re.sub(cls2,'',qb_html_3)
qb_html_3=re.sub(ab1,'',qb_html_3)
qb_html_3=re.sub(ab2,'',qb_html_3)
qb_html_3=re.sub(ab3,'',qb_html_3)
qb_html_3=re.sub(ab4,'',qb_html_3)
qb_html_3=re.sub(ab5,'',qb_html_3)
qb_html_3=re.sub(ab6,'',qb_html_3)
qb_html_3=re.sub(ab7,'',qb_html_3)
qb_html_3=re.sub(ab8_3,'',qb_html_3)

#Week 4
qb_url_4 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=4&pos=qb&col=FPTS&ccs=96662'
qb_html_4 = requests.get(qb_url_4).text

qb_html_4=re.sub(header,'',qb_html_4)
qb_html_4=re.sub(cls,'',qb_html_4)
qb_html_4=re.sub(cls1,'',qb_html_4)
qb_html_4=re.sub(cls2,'',qb_html_4)
qb_html_4=re.sub(ab1,'',qb_html_4)
qb_html_4=re.sub(ab2,'',qb_html_4)
qb_html_4=re.sub(ab3,'',qb_html_4)
qb_html_4=re.sub(ab4,'',qb_html_4)
qb_html_4=re.sub(ab5,'',qb_html_4)
qb_html_4=re.sub(ab6,'',qb_html_4)
qb_html_4=re.sub(ab7,'',qb_html_4)
qb_html_4=re.sub(ab8_4,'',qb_html_4)

#Week 5
qb_url_5 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=5&pos=qb&col=FPTS&ccs=96662'
qb_html_5 = requests.get(qb_url_5).text

qb_html_5=re.sub(header,'',qb_html_5)
qb_html_5=re.sub(cls,'',qb_html_5)
qb_html_5=re.sub(cls1,'',qb_html_5)
qb_html_5=re.sub(cls2,'',qb_html_5)
qb_html_5=re.sub(ab1,'',qb_html_5)
qb_html_5=re.sub(ab2,'',qb_html_5)
qb_html_5=re.sub(ab3,'',qb_html_5)
qb_html_5=re.sub(ab4,'',qb_html_5)
qb_html_5=re.sub(ab5,'',qb_html_5)
qb_html_5=re.sub(ab6,'',qb_html_5)
qb_html_5=re.sub(ab7,'',qb_html_5)
qb_html_5=re.sub(ab8_5,'',qb_html_5)

#Week 6
qb_url_6 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=6&pos=qb&col=FPTS&ccs=96662'
qb_html_6 = requests.get(qb_url_6).text

qb_html_6=re.sub(header,'',qb_html_6)
qb_html_6=re.sub(cls,'',qb_html_6)
qb_html_6=re.sub(cls1,'',qb_html_6)
qb_html_6=re.sub(cls2,'',qb_html_6)
qb_html_6=re.sub(ab1,'',qb_html_6)
qb_html_6=re.sub(ab2,'',qb_html_6)
qb_html_6=re.sub(ab3,'',qb_html_6)
qb_html_6=re.sub(ab4,'',qb_html_6)
qb_html_6=re.sub(ab5,'',qb_html_6)
qb_html_6=re.sub(ab6,'',qb_html_6)
qb_html_6=re.sub(ab7,'',qb_html_6)
qb_html_6=re.sub(ab8_6,'',qb_html_6)

#Week 7
qb_url_7 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=7&pos=qb&col=FPTS&ccs=96662'
qb_html_7 = requests.get(qb_url_7).text

qb_html_7=re.sub(header,'',qb_html_7)
qb_html_7=re.sub(cls,'',qb_html_7)
qb_html_7=re.sub(cls1,'',qb_html_7)
qb_html_7=re.sub(cls2,'',qb_html_7)
qb_html_7=re.sub(ab1,'',qb_html_7)
qb_html_7=re.sub(ab2,'',qb_html_7)
qb_html_7=re.sub(ab3,'',qb_html_7)
qb_html_7=re.sub(ab4,'',qb_html_7)
qb_html_7=re.sub(ab5,'',qb_html_7)
qb_html_7=re.sub(ab6,'',qb_html_7)
qb_html_7=re.sub(ab7,'',qb_html_7)
qb_html_7=re.sub(ab8_7,'',qb_html_7)

#Week 8
qb_url_8 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=8&pos=qb&col=FPTS&ccs=96662'
qb_html_8 = requests.get(qb_url_8).text

qb_html_8=re.sub(header,'',qb_html_8)
qb_html_8=re.sub(cls,'',qb_html_8)
qb_html_8=re.sub(cls1,'',qb_html_8)
qb_html_8=re.sub(cls2,'',qb_html_8)
qb_html_8=re.sub(ab1,'',qb_html_8)
qb_html_8=re.sub(ab2,'',qb_html_8)
qb_html_8=re.sub(ab3,'',qb_html_8)
qb_html_8=re.sub(ab4,'',qb_html_8)
qb_html_8=re.sub(ab5,'',qb_html_8)
qb_html_8=re.sub(ab6,'',qb_html_8)
qb_html_8=re.sub(ab7,'',qb_html_8)
qb_html_8=re.sub(ab8_8,'',qb_html_8)

#Week 9
qb_url_9 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=9&pos=qb&col=FPTS&ccs=96662'
qb_html_9 = requests.get(qb_url_9).text

qb_html_9=re.sub(header,'',qb_html_9)
qb_html_9=re.sub(cls,'',qb_html_9)
qb_html_9=re.sub(cls1,'',qb_html_9)
qb_html_9=re.sub(cls2,'',qb_html_9)
qb_html_9=re.sub(ab1,'',qb_html_9)
qb_html_9=re.sub(ab2,'',qb_html_9)
qb_html_9=re.sub(ab3,'',qb_html_9)
qb_html_9=re.sub(ab4,'',qb_html_9)
qb_html_9=re.sub(ab5,'',qb_html_9)
qb_html_9=re.sub(ab6,'',qb_html_9)
qb_html_9=re.sub(ab7,'',qb_html_9)
qb_html_9=re.sub(ab8_9,'',qb_html_9)

#Week 10
qb_url_10 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=10&pos=qb&col=FPTS&ccs=96662'
qb_html_10 = requests.get(qb_url_10).text

qb_html_10=re.sub(header,'',qb_html_10)
qb_html_10=re.sub(cls,'',qb_html_10)
qb_html_10=re.sub(cls1,'',qb_html_10)
qb_html_10=re.sub(cls2,'',qb_html_10)
qb_html_10=re.sub(ab1,'',qb_html_10)
qb_html_10=re.sub(ab2,'',qb_html_10)
qb_html_10=re.sub(ab3,'',qb_html_10)
qb_html_10=re.sub(ab4,'',qb_html_10)
qb_html_10=re.sub(ab5,'',qb_html_10)
qb_html_10=re.sub(ab6,'',qb_html_10)
qb_html_10=re.sub(ab7,'',qb_html_10)
qb_html_10=re.sub(ab8_10,'',qb_html_10)

#Week 11
qb_url_11 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=11&pos=qb&col=FPTS&ccs=96662'
qb_html_11 = requests.get(qb_url_11).text

qb_html_11=re.sub(header,'',qb_html_11)
qb_html_11=re.sub(cls,'',qb_html_11)
qb_html_11=re.sub(cls1,'',qb_html_11)
qb_html_11=re.sub(cls2,'',qb_html_11)
qb_html_11=re.sub(ab1,'',qb_html_11)
qb_html_11=re.sub(ab2,'',qb_html_11)
qb_html_11=re.sub(ab3,'',qb_html_11)
qb_html_11=re.sub(ab4,'',qb_html_11)
qb_html_11=re.sub(ab5,'',qb_html_11)
qb_html_11=re.sub(ab6,'',qb_html_11)
qb_html_11=re.sub(ab7,'',qb_html_11)
qb_html_11=re.sub(ab8_11,'',qb_html_11)

### Wide Recievers

In [ ]:
#Week 1
wr_url_1 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=1&pos=wr&col=FPTS&ccs=96662'
wr_html_1 = requests.get(wr_url_1).text

wr_html_1=re.sub(header,'',wr_html_1)
wr_html_1=re.sub(cls,'',wr_html_1)
wr_html_1=re.sub(cls1,'',wr_html_1)
wr_html_1=re.sub(cls2,'',wr_html_1)
wr_html_1=re.sub(ab1,'',wr_html_1)
wr_html_1=re.sub(ab2,'',wr_html_1)
wr_html_1=re.sub(ab3,'',wr_html_1)
wr_html_1=re.sub(ab4,'',wr_html_1)
wr_html_1=re.sub(ab5,'',wr_html_1)
wr_html_1=re.sub(ab6,'',wr_html_1)
wr_html_1=re.sub(ab7,'',wr_html_1)
wr_html_1=re.sub(ab8_1,'',wr_html_1)

#Week 2
wr_url_2 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=2&pos=wr&col=FPTS&ccs=96662'
wr_html_2 = requests.get(wr_url_2).text

wr_html_2=re.sub(header,'',wr_html_2)
wr_html_2=re.sub(cls,'',wr_html_2)
wr_html_2=re.sub(cls1,'',wr_html_2)
wr_html_2=re.sub(cls2,'',wr_html_2)
wr_html_2=re.sub(ab1,'',wr_html_2)
wr_html_2=re.sub(ab2,'',wr_html_2)
wr_html_2=re.sub(ab3,'',wr_html_2)
wr_html_2=re.sub(ab4,'',wr_html_2)
wr_html_2=re.sub(ab5,'',wr_html_2)
wr_html_2=re.sub(ab6,'',wr_html_2)
wr_html_2=re.sub(ab7,'',wr_html_2)
wr_html_2=re.sub(ab8_2,'',wr_html_2)

#Week 3
wr_url_3 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=3&pos=wr&col=FPTS&ccs=96662'
wr_html_3 = requests.get(wr_url_3).text

wr_html_3=re.sub(header,'',wr_html_3)
wr_html_3=re.sub(cls,'',wr_html_3)
wr_html_3=re.sub(cls1,'',wr_html_3)
wr_html_3=re.sub(cls2,'',wr_html_3)
wr_html_3=re.sub(ab1,'',wr_html_3)
wr_html_3=re.sub(ab2,'',wr_html_3)
wr_html_3=re.sub(ab3,'',wr_html_3)
wr_html_3=re.sub(ab4,'',wr_html_3)
wr_html_3=re.sub(ab5,'',wr_html_3)
wr_html_3=re.sub(ab6,'',wr_html_3)
wr_html_3=re.sub(ab7,'',wr_html_3)
wr_html_3=re.sub(ab8_3,'',wr_html_3)

#Week 4
wr_url_4 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=4&pos=wr&col=FPTS&ccs=96662'
wr_html_4 = requests.get(wr_url_4).text

wr_html_4=re.sub(header,'',wr_html_4)
wr_html_4=re.sub(cls,'',wr_html_4)
wr_html_4=re.sub(cls1,'',wr_html_4)
wr_html_4=re.sub(cls2,'',wr_html_4)
wr_html_4=re.sub(ab1,'',wr_html_4)
wr_html_4=re.sub(ab2,'',wr_html_4)
wr_html_4=re.sub(ab3,'',wr_html_4)
wr_html_4=re.sub(ab4,'',wr_html_4)
wr_html_4=re.sub(ab5,'',wr_html_4)
wr_html_4=re.sub(ab6,'',wr_html_4)
wr_html_4=re.sub(ab7,'',wr_html_4)
wr_html_4=re.sub(ab8_4,'',wr_html_4)

#Week 5
wr_url_5 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=5&pos=wr&col=FPTS&ccs=96662'
wr_html_5 = requests.get(wr_url_5).text

wr_html_5=re.sub(header,'',wr_html_5)
wr_html_5=re.sub(cls,'',wr_html_5)
wr_html_5=re.sub(cls1,'',wr_html_5)
wr_html_5=re.sub(cls2,'',wr_html_5)
wr_html_5=re.sub(ab1,'',wr_html_5)
wr_html_5=re.sub(ab2,'',wr_html_5)
wr_html_5=re.sub(ab3,'',wr_html_5)
wr_html_5=re.sub(ab4,'',wr_html_5)
wr_html_5=re.sub(ab5,'',wr_html_5)
wr_html_5=re.sub(ab6,'',wr_html_5)
wr_html_5=re.sub(ab7,'',wr_html_5)
wr_html_5=re.sub(ab8_5,'',wr_html_5)

#Week 6
wr_url_6 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=6&pos=wr&col=FPTS&ccs=96662'
wr_html_6 = requests.get(wr_url_6).text

wr_html_6=re.sub(header,'',wr_html_6)
wr_html_6=re.sub(cls,'',wr_html_6)
wr_html_6=re.sub(cls1,'',wr_html_6)
wr_html_6=re.sub(cls2,'',wr_html_6)
wr_html_6=re.sub(ab1,'',wr_html_6)
wr_html_6=re.sub(ab2,'',wr_html_6)
wr_html_6=re.sub(ab3,'',wr_html_6)
wr_html_6=re.sub(ab4,'',wr_html_6)
wr_html_6=re.sub(ab5,'',wr_html_6)
wr_html_6=re.sub(ab6,'',wr_html_6)
wr_html_6=re.sub(ab7,'',wr_html_6)
wr_html_6=re.sub(ab8_6,'',wr_html_6)

#Week 7
wr_url_7 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=7&pos=wr&col=FPTS&ccs=96662'
wr_html_7 = requests.get(wr_url_7).text

wr_html_7=re.sub(header,'',wr_html_7)
wr_html_7=re.sub(cls,'',wr_html_7)
wr_html_7=re.sub(cls1,'',wr_html_7)
wr_html_7=re.sub(cls2,'',wr_html_7)
wr_html_7=re.sub(ab1,'',wr_html_7)
wr_html_7=re.sub(ab2,'',wr_html_7)
wr_html_7=re.sub(ab3,'',wr_html_7)
wr_html_7=re.sub(ab4,'',wr_html_7)
wr_html_7=re.sub(ab5,'',wr_html_7)
wr_html_7=re.sub(ab6,'',wr_html_7)
wr_html_7=re.sub(ab7,'',wr_html_7)
wr_html_7=re.sub(ab8_7,'',wr_html_7)

#Week 8
wr_url_8 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=8&pos=wr&col=FPTS&ccs=96662'
wr_html_8 = requests.get(wr_url_8).text

wr_html_8=re.sub(header,'',wr_html_8)
wr_html_8=re.sub(cls,'',wr_html_8)
wr_html_8=re.sub(cls1,'',wr_html_8)
wr_html_8=re.sub(cls2,'',wr_html_8)
wr_html_8=re.sub(ab1,'',wr_html_8)
wr_html_8=re.sub(ab2,'',wr_html_8)
wr_html_8=re.sub(ab3,'',wr_html_8)
wr_html_8=re.sub(ab4,'',wr_html_8)
wr_html_8=re.sub(ab5,'',wr_html_8)
wr_html_8=re.sub(ab6,'',wr_html_8)
wr_html_8=re.sub(ab7,'',wr_html_8)
wr_html_8=re.sub(ab8_8,'',wr_html_8)

#Week 9
wr_url_9 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=9&pos=wr&col=FPTS&ccs=96662'
wr_html_9 = requests.get(wr_url_9).text

wr_html_9=re.sub(header,'',wr_html_9)
wr_html_9=re.sub(cls,'',wr_html_9)
wr_html_9=re.sub(cls1,'',wr_html_9)
wr_html_9=re.sub(cls2,'',wr_html_9)
wr_html_9=re.sub(ab1,'',wr_html_9)
wr_html_9=re.sub(ab2,'',wr_html_9)
wr_html_9=re.sub(ab3,'',wr_html_9)
wr_html_9=re.sub(ab4,'',wr_html_9)
wr_html_9=re.sub(ab5,'',wr_html_9)
wr_html_9=re.sub(ab6,'',wr_html_9)
wr_html_9=re.sub(ab7,'',wr_html_9)
wr_html_9=re.sub(ab8_9,'',wr_html_9)

#Week 10
wr_url_10 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=10&pos=wr&col=FPTS&ccs=96662'
wr_html_10 = requests.get(wr_url_10).text

wr_html_10=re.sub(header,'',wr_html_10)
wr_html_10=re.sub(cls,'',wr_html_10)
wr_html_10=re.sub(cls1,'',wr_html_10)
wr_html_10=re.sub(cls2,'',wr_html_10)
wr_html_10=re.sub(ab1,'',wr_html_10)
wr_html_10=re.sub(ab2,'',wr_html_10)
wr_html_10=re.sub(ab3,'',wr_html_10)
wr_html_10=re.sub(ab4,'',wr_html_10)
wr_html_10=re.sub(ab5,'',wr_html_10)
wr_html_10=re.sub(ab6,'',wr_html_10)
wr_html_10=re.sub(ab7,'',wr_html_10)
wr_html_10=re.sub(ab8_10,'',wr_html_10)

#Week 11
wr_url_11 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=11&pos=wr&col=FPTS&ccs=96662'
wr_html_11 = requests.get(wr_url_11).text

wr_html_11=re.sub(header,'',wr_html_11)
wr_html_11=re.sub(cls,'',wr_html_11)
wr_html_11=re.sub(cls1,'',wr_html_11)
wr_html_11=re.sub(cls2,'',wr_html_11)
wr_html_11=re.sub(ab1,'',wr_html_11)
wr_html_11=re.sub(ab2,'',wr_html_11)
wr_html_11=re.sub(ab3,'',wr_html_11)
wr_html_11=re.sub(ab4,'',wr_html_11)
wr_html_11=re.sub(ab5,'',wr_html_11)
wr_html_11=re.sub(ab6,'',wr_html_11)
wr_html_11=re.sub(ab7,'',wr_html_11)
wr_html_11=re.sub(ab8_11,'',wr_html_11)

### Tight Ends

In [ ]:
#Week 1
te_url_1 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=1&pos=te&col=FPTS&ccs=96662'
te_html_1 = requests.get(te_url_1).text

te_html_1=re.sub(header,'',te_html_1)
te_html_1=re.sub(cls,'',te_html_1)
te_html_1=re.sub(cls1,'',te_html_1)
te_html_1=re.sub(cls2,'',te_html_1)
te_html_1=re.sub(ab1,'',te_html_1)
te_html_1=re.sub(ab2,'',te_html_1)
te_html_1=re.sub(ab3,'',te_html_1)
te_html_1=re.sub(ab4,'',te_html_1)
te_html_1=re.sub(ab5,'',te_html_1)
te_html_1=re.sub(ab6,'',te_html_1)
te_html_1=re.sub(ab7,'',te_html_1)
te_html_1=re.sub(ab8_1,'',te_html_1)

#Week 2
te_url_2 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=2&pos=te&col=FPTS&ccs=96662'
te_html_2 = requests.get(te_url_2).text

te_html_2=re.sub(header,'',te_html_2)
te_html_2=re.sub(cls,'',te_html_2)
te_html_2=re.sub(cls1,'',te_html_2)
te_html_2=re.sub(cls2,'',te_html_2)
te_html_2=re.sub(ab1,'',te_html_2)
te_html_2=re.sub(ab2,'',te_html_2)
te_html_2=re.sub(ab3,'',te_html_2)
te_html_2=re.sub(ab4,'',te_html_2)
te_html_2=re.sub(ab5,'',te_html_2)
te_html_2=re.sub(ab6,'',te_html_2)
te_html_2=re.sub(ab7,'',te_html_2)
te_html_2=re.sub(ab8_2,'',te_html_2)

#Week 3
te_url_3 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=3&pos=te&col=FPTS&ccs=96662'
te_html_3 = requests.get(te_url_3).text

te_html_3=re.sub(header,'',te_html_3)
te_html_3=re.sub(cls,'',te_html_3)
te_html_3=re.sub(cls1,'',te_html_3)
te_html_3=re.sub(cls2,'',te_html_3)
te_html_3=re.sub(ab1,'',te_html_3)
te_html_3=re.sub(ab2,'',te_html_3)
te_html_3=re.sub(ab3,'',te_html_3)
te_html_3=re.sub(ab4,'',te_html_3)
te_html_3=re.sub(ab5,'',te_html_3)
te_html_3=re.sub(ab6,'',te_html_3)
te_html_3=re.sub(ab7,'',te_html_3)
te_html_3=re.sub(ab8_3,'',te_html_3)

#Week 4
te_url_4 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=4&pos=te&col=FPTS&ccs=96662'
te_html_4 = requests.get(te_url_4).text

te_html_4=re.sub(header,'',te_html_4)
te_html_4=re.sub(cls,'',te_html_4)
te_html_4=re.sub(cls1,'',te_html_4)
te_html_4=re.sub(cls2,'',te_html_4)
te_html_4=re.sub(ab1,'',te_html_4)
te_html_4=re.sub(ab2,'',te_html_4)
te_html_4=re.sub(ab3,'',te_html_4)
te_html_4=re.sub(ab4,'',te_html_4)
te_html_4=re.sub(ab5,'',te_html_4)
te_html_4=re.sub(ab6,'',te_html_4)
te_html_4=re.sub(ab7,'',te_html_4)
te_html_4=re.sub(ab8_4,'',te_html_4)

#Week 5
te_url_5 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=5&pos=te&col=FPTS&ccs=96662'
te_html_5 = requests.get(te_url_5).text

te_html_5=re.sub(header,'',te_html_5)
te_html_5=re.sub(cls,'',te_html_5)
te_html_5=re.sub(cls1,'',te_html_5)
te_html_5=re.sub(cls2,'',te_html_5)
te_html_5=re.sub(ab1,'',te_html_5)
te_html_5=re.sub(ab2,'',te_html_5)
te_html_5=re.sub(ab3,'',te_html_5)
te_html_5=re.sub(ab4,'',te_html_5)
te_html_5=re.sub(ab5,'',te_html_5)
te_html_5=re.sub(ab6,'',te_html_5)
te_html_5=re.sub(ab7,'',te_html_5)
te_html_5=re.sub(ab8_5,'',te_html_5)

#Week 6
te_url_6 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=6&pos=te&col=FPTS&ccs=96662'
te_html_6 = requests.get(te_url_6).text

te_html_6=re.sub(header,'',te_html_6)
te_html_6=re.sub(cls,'',te_html_6)
te_html_6=re.sub(cls1,'',te_html_6)
te_html_6=re.sub(cls2,'',te_html_6)
te_html_6=re.sub(ab1,'',te_html_6)
te_html_6=re.sub(ab2,'',te_html_6)
te_html_6=re.sub(ab3,'',te_html_6)
te_html_6=re.sub(ab4,'',te_html_6)
te_html_6=re.sub(ab5,'',te_html_6)
te_html_6=re.sub(ab6,'',te_html_6)
te_html_6=re.sub(ab7,'',te_html_6)
te_html_6=re.sub(ab8_6,'',te_html_6)

#Week 7
te_url_7 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=7&pos=te&col=FPTS&ccs=96662'
te_html_7 = requests.get(te_url_7).text

te_html_7=re.sub(header,'',te_html_7)
te_html_7=re.sub(cls,'',te_html_7)
te_html_7=re.sub(cls1,'',te_html_7)
te_html_7=re.sub(cls2,'',te_html_7)
te_html_7=re.sub(ab1,'',te_html_7)
te_html_7=re.sub(ab2,'',te_html_7)
te_html_7=re.sub(ab3,'',te_html_7)
te_html_7=re.sub(ab4,'',te_html_7)
te_html_7=re.sub(ab5,'',te_html_7)
te_html_7=re.sub(ab6,'',te_html_7)
te_html_7=re.sub(ab7,'',te_html_7)
te_html_7=re.sub(ab8_7,'',te_html_7)

#Week 8
te_url_8 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=8&pos=te&col=FPTS&ccs=96662'
te_html_8 = requests.get(te_url_8).text

te_html_8=re.sub(header,'',te_html_8)
te_html_8=re.sub(cls,'',te_html_8)
te_html_8=re.sub(cls1,'',te_html_8)
te_html_8=re.sub(cls2,'',te_html_8)
te_html_8=re.sub(ab1,'',te_html_8)
te_html_8=re.sub(ab2,'',te_html_8)
te_html_8=re.sub(ab3,'',te_html_8)
te_html_8=re.sub(ab4,'',te_html_8)
te_html_8=re.sub(ab5,'',te_html_8)
te_html_8=re.sub(ab6,'',te_html_8)
te_html_8=re.sub(ab7,'',te_html_8)
te_html_8=re.sub(ab8_8,'',te_html_8)

#Week 9
te_url_9 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=9&pos=te&col=FPTS&ccs=96662'
te_html_9 = requests.get(te_url_9).text

te_html_9=re.sub(header,'',te_html_9)
te_html_9=re.sub(cls,'',te_html_9)
te_html_9=re.sub(cls1,'',te_html_9)
te_html_9=re.sub(cls2,'',te_html_9)
te_html_9=re.sub(ab1,'',te_html_9)
te_html_9=re.sub(ab2,'',te_html_9)
te_html_9=re.sub(ab3,'',te_html_9)
te_html_9=re.sub(ab4,'',te_html_9)
te_html_9=re.sub(ab5,'',te_html_9)
te_html_9=re.sub(ab6,'',te_html_9)
te_html_9=re.sub(ab7,'',te_html_9)
te_html_9=re.sub(ab8_9,'',te_html_9)

#Week 10
te_url_10 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=10&pos=te&col=FPTS&ccs=96662'
te_html_10 = requests.get(te_url_10).text

te_html_10=re.sub(header,'',te_html_10)
te_html_10=re.sub(cls,'',te_html_10)
te_html_10=re.sub(cls1,'',te_html_10)
te_html_10=re.sub(cls2,'',te_html_10)
te_html_10=re.sub(ab1,'',te_html_10)
te_html_10=re.sub(ab2,'',te_html_10)
te_html_10=re.sub(ab3,'',te_html_10)
te_html_10=re.sub(ab4,'',te_html_10)
te_html_10=re.sub(ab5,'',te_html_10)
te_html_10=re.sub(ab6,'',te_html_10)
te_html_10=re.sub(ab7,'',te_html_10)
te_html_10=re.sub(ab8_10,'',te_html_10)

#Week 11
te_url_11 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=11&pos=te&col=FPTS&ccs=96662'
te_html_11 = requests.get(te_url_11).text

te_html_11=re.sub(header,'',te_html_11)
te_html_11=re.sub(cls,'',te_html_11)
te_html_11=re.sub(cls1,'',te_html_11)
te_html_11=re.sub(cls2,'',te_html_11)
te_html_11=re.sub(ab1,'',te_html_11)
te_html_11=re.sub(ab2,'',te_html_11)
te_html_11=re.sub(ab3,'',te_html_11)
te_html_11=re.sub(ab4,'',te_html_11)
te_html_11=re.sub(ab5,'',te_html_11)
te_html_11=re.sub(ab6,'',te_html_11)
te_html_11=re.sub(ab7,'',te_html_11)
te_html_11=re.sub(ab8_11,'',te_html_11)

### Defense

In [ ]:
#Week 1
df_url_1 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=1&pos=df&col=FPTS&ccs=96662'
df_html_1 = requests.get(df_url_1).text

df_html_1=re.sub(header,'',df_html_1)
df_html_1=re.sub(cls,'',df_html_1)
df_html_1=re.sub(cls1,'',df_html_1)
df_html_1=re.sub(cls2,'',df_html_1)
df_html_1=re.sub(ab1,'',df_html_1)
df_html_1=re.sub(ab2,'',df_html_1)
df_html_1=re.sub(ab3,'',df_html_1)
df_html_1=re.sub(ab4,'',df_html_1)
df_html_1=re.sub(ab5,'',df_html_1)
df_html_1=re.sub(ab6,'',df_html_1)
df_html_1=re.sub(ab7,'',df_html_1)
df_html_1=re.sub(ab8_1,'',df_html_1)

#Week 2
df_url_2 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=2&pos=df&col=FPTS&ccs=96662'
df_html_2 = requests.get(df_url_2).text

df_html_2=re.sub(header,'',df_html_2)
df_html_2=re.sub(cls,'',df_html_2)
df_html_2=re.sub(cls1,'',df_html_2)
df_html_2=re.sub(cls2,'',df_html_2)
df_html_2=re.sub(ab1,'',df_html_2)
df_html_2=re.sub(ab2,'',df_html_2)
df_html_2=re.sub(ab3,'',df_html_2)
df_html_2=re.sub(ab4,'',df_html_2)
df_html_2=re.sub(ab5,'',df_html_2)
df_html_2=re.sub(ab6,'',df_html_2)
df_html_2=re.sub(ab7,'',df_html_2)
df_html_2=re.sub(ab8_2,'',df_html_2)

#Week 3
df_url_3 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=3&pos=df&col=FPTS&ccs=96662'
df_html_3 = requests.get(df_url_3).text

df_html_3=re.sub(header,'',df_html_3)
df_html_3=re.sub(cls,'',df_html_3)
df_html_3=re.sub(cls1,'',df_html_3)
df_html_3=re.sub(cls2,'',df_html_3)
df_html_3=re.sub(ab1,'',df_html_3)
df_html_3=re.sub(ab2,'',df_html_3)
df_html_3=re.sub(ab3,'',df_html_3)
df_html_3=re.sub(ab4,'',df_html_3)
df_html_3=re.sub(ab5,'',df_html_3)
df_html_3=re.sub(ab6,'',df_html_3)
df_html_3=re.sub(ab7,'',df_html_3)
df_html_3=re.sub(ab8_3,'',df_html_3)

#Week 4
df_url_4 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=4&pos=df&col=FPTS&ccs=96662'
df_html_4 = requests.get(df_url_4).text

df_html_4=re.sub(header,'',df_html_4)
df_html_4=re.sub(cls,'',df_html_4)
df_html_4=re.sub(cls1,'',df_html_4)
df_html_4=re.sub(cls2,'',df_html_4)
df_html_4=re.sub(ab1,'',df_html_4)
df_html_4=re.sub(ab2,'',df_html_4)
df_html_4=re.sub(ab3,'',df_html_4)
df_html_4=re.sub(ab4,'',df_html_4)
df_html_4=re.sub(ab5,'',df_html_4)
df_html_4=re.sub(ab6,'',df_html_4)
df_html_4=re.sub(ab7,'',df_html_4)
df_html_4=re.sub(ab8_4,'',df_html_4)

#Week 5
df_url_5 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=5&pos=df&col=FPTS&ccs=96662'
df_html_5 = requests.get(df_url_5).text

df_html_5=re.sub(header,'',df_html_5)
df_html_5=re.sub(cls,'',df_html_5)
df_html_5=re.sub(cls1,'',df_html_5)
df_html_5=re.sub(cls2,'',df_html_5)
df_html_5=re.sub(ab1,'',df_html_5)
df_html_5=re.sub(ab2,'',df_html_5)
df_html_5=re.sub(ab3,'',df_html_5)
df_html_5=re.sub(ab4,'',df_html_5)
df_html_5=re.sub(ab5,'',df_html_5)
df_html_5=re.sub(ab6,'',df_html_5)
df_html_5=re.sub(ab7,'',df_html_5)
df_html_5=re.sub(ab8_5,'',df_html_5)

#Week 6
df_url_6 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=6&pos=df&col=FPTS&ccs=96662'
df_html_6 = requests.get(df_url_6).text

df_html_6=re.sub(header,'',df_html_6)
df_html_6=re.sub(cls,'',df_html_6)
df_html_6=re.sub(cls1,'',df_html_6)
df_html_6=re.sub(cls2,'',df_html_6)
df_html_6=re.sub(ab1,'',df_html_6)
df_html_6=re.sub(ab2,'',df_html_6)
df_html_6=re.sub(ab3,'',df_html_6)
df_html_6=re.sub(ab4,'',df_html_6)
df_html_6=re.sub(ab5,'',df_html_6)
df_html_6=re.sub(ab6,'',df_html_6)
df_html_6=re.sub(ab7,'',df_html_6)
df_html_6=re.sub(ab8_6,'',df_html_6)

#Week 7
df_url_7 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=7&pos=df&col=FPTS&ccs=96662'
df_html_7 = requests.get(df_url_7).text

df_html_7=re.sub(header,'',df_html_7)
df_html_7=re.sub(cls,'',df_html_7)
df_html_7=re.sub(cls1,'',df_html_7)
df_html_7=re.sub(cls2,'',df_html_7)
df_html_7=re.sub(ab1,'',df_html_7)
df_html_7=re.sub(ab2,'',df_html_7)
df_html_7=re.sub(ab3,'',df_html_7)
df_html_7=re.sub(ab4,'',df_html_7)
df_html_7=re.sub(ab5,'',df_html_7)
df_html_7=re.sub(ab6,'',df_html_7)
df_html_7=re.sub(ab7,'',df_html_7)
df_html_7=re.sub(ab8_7,'',df_html_7)

#Week 8
df_url_8 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=8&pos=df&col=FPTS&ccs=96662'
df_html_8 = requests.get(df_url_8).text

df_html_8=re.sub(header,'',df_html_8)
df_html_8=re.sub(cls,'',df_html_8)
df_html_8=re.sub(cls1,'',df_html_8)
df_html_8=re.sub(cls2,'',df_html_8)
df_html_8=re.sub(ab1,'',df_html_8)
df_html_8=re.sub(ab2,'',df_html_8)
df_html_8=re.sub(ab3,'',df_html_8)
df_html_8=re.sub(ab4,'',df_html_8)
df_html_8=re.sub(ab5,'',df_html_8)
df_html_8=re.sub(ab6,'',df_html_8)
df_html_8=re.sub(ab7,'',df_html_8)
df_html_8=re.sub(ab8_8,'',df_html_8)

#Week 9
df_url_9 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=9&pos=df&col=FPTS&ccs=96662'
df_html_9 = requests.get(df_url_9).text

df_html_9=re.sub(header,'',df_html_9)
df_html_9=re.sub(cls,'',df_html_9)
df_html_9=re.sub(cls1,'',df_html_9)
df_html_9=re.sub(cls2,'',df_html_9)
df_html_9=re.sub(ab1,'',df_html_9)
df_html_9=re.sub(ab2,'',df_html_9)
df_html_9=re.sub(ab3,'',df_html_9)
df_html_9=re.sub(ab4,'',df_html_9)
df_html_9=re.sub(ab5,'',df_html_9)
df_html_9=re.sub(ab6,'',df_html_9)
df_html_9=re.sub(ab7,'',df_html_9)
df_html_9=re.sub(ab8_9,'',df_html_9)

#Week 10
df_url_10 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=10&pos=df&col=FPTS&ccs=96662'
df_html_10 = requests.get(df_url_10).text

df_html_10=re.sub(header,'',df_html_10)
df_html_10=re.sub(cls,'',df_html_10)
df_html_10=re.sub(cls1,'',df_html_10)
df_html_10=re.sub(cls2,'',df_html_10)
df_html_10=re.sub(ab1,'',df_html_10)
df_html_10=re.sub(ab2,'',df_html_10)
df_html_10=re.sub(ab3,'',df_html_10)
df_html_10=re.sub(ab4,'',df_html_10)
df_html_10=re.sub(ab5,'',df_html_10)
df_html_10=re.sub(ab6,'',df_html_10)
df_html_10=re.sub(ab7,'',df_html_10)
df_html_10=re.sub(ab8_10,'',df_html_10)

#Week 11
df_url_11 = 'http://www.thehuddle.com/stats/2015/plays_weekly.php?week=11&pos=df&col=FPTS&ccs=96662'
df_html_11 = requests.get(df_url_11).text

df_html_11=re.sub(header,'',df_html_11)
df_html_11=re.sub(cls,'',df_html_11)
df_html_11=re.sub(cls1,'',df_html_11)
df_html_11=re.sub(cls2,'',df_html_11)
df_html_11=re.sub(ab1,'',df_html_11)
df_html_11=re.sub(ab2,'',df_html_11)
df_html_11=re.sub(ab3,'',df_html_11)
df_html_11=re.sub(ab4,'',df_html_11)
df_html_11=re.sub(ab5,'',df_html_11)
df_html_11=re.sub(ab6,'',df_html_11)
df_html_11=re.sub(ab7,'',df_html_11)
df_html_11=re.sub(ab8_11,'',df_html_11)

## Run Functions on Cleaned Data:

### Running Backs

In [ ]:
#Week 1
rb_table_1 = get_table(rb_html_1)
rb_by_type_1 = defaultdict(list)  
for tbl in rb_table_1:
    rb_by_type_1[table_type(tbl)].append(tbl)
rb_result_1 = get_stats_rb(rb_by_type_1['PLAYER&pos;=rb&week;"&gt;PLAYER'])

#Week 2
rb_table_2 = get_table(rb_html_2)   
rb_by_type_2 = defaultdict(list)  
for tbl in rb_table_2:
    rb_by_type_2[table_type(tbl)].append(tbl)
rb_result_2 = get_stats_rb(rb_by_type_2['PLAYER&pos;=rb&week;"&gt;PLAYER'])

#Week 3
rb_table_3 = get_table(rb_html_3)   
rb_by_type_3 = defaultdict(list)  
for tbl in rb_table_3:
    rb_by_type_3[table_type(tbl)].append(tbl)
rb_result_3 = get_stats_rb(rb_by_type_3['PLAYER&pos;=rb&week;"&gt;PLAYER'])

#Week 4
rb_table_4 = get_table(rb_html_4)   
rb_by_type_4 = defaultdict(list)  
for tbl in rb_table_4:
    rb_by_type_4[table_type(tbl)].append(tbl)
rb_result_4 = get_stats_rb(rb_by_type_4['PLAYER&pos;=rb&week;"&gt;PLAYER'])

#Week 5
rb_table_5 = get_table(rb_html_5)   
rb_by_type_5 = defaultdict(list)  
for tbl in rb_table_5:
    rb_by_type_5[table_type(tbl)].append(tbl)
rb_result_5 = get_stats_rb(rb_by_type_5['PLAYER&pos;=rb&week;"&gt;PLAYER'])

#Week 6
rb_table_6 = get_table(rb_html_6)   
rb_by_type_6 = defaultdict(list)  
for tbl in rb_table_6:
    rb_by_type_6[table_type(tbl)].append(tbl)
rb_result_6 = get_stats_rb(rb_by_type_6['PLAYER&pos;=rb&week;"&gt;PLAYER'])

#Week 7
rb_table_7 = get_table(rb_html_7)   
rb_by_type_7 = defaultdict(list)  
for tbl in rb_table_7:
    rb_by_type_7[table_type(tbl)].append(tbl)
rb_result_7 = get_stats_rb(rb_by_type_7['PLAYER&pos;=rb&week;"&gt;PLAYER'])

#Week 8
rb_table_8 = get_table(rb_html_8)   
rb_by_type_8 = defaultdict(list)  
for tbl in rb_table_8:
    rb_by_type_8[table_type(tbl)].append(tbl)
rb_result_8 = get_stats_rb(rb_by_type_8['PLAYER&pos;=rb&week;"&gt;PLAYER'])

#Week 9
rb_table_9 = get_table(rb_html_9)   
rb_by_type_9 = defaultdict(list)  
for tbl in rb_table_9:
    rb_by_type_9[table_type(tbl)].append(tbl)
rb_result_9 = get_stats_rb(rb_by_type_9['PLAYER&pos;=rb&week;"&gt;PLAYER'])

#Week 10
rb_table_10 = get_table(rb_html_10)   
rb_by_type_10 = defaultdict(list)  
for tbl in rb_table_10:
    rb_by_type_10[table_type(tbl)].append(tbl)
rb_result_10 = get_stats_rb(rb_by_type_10['PLAYER&pos;=rb&week;"&gt;PLAYER'])

#Week 11
rb_table_11 = get_table(rb_html_11)   
rb_by_type_11 = defaultdict(list)  
for tbl in rb_table_11:
    rb_by_type_11[table_type(tbl)].append(tbl)
rb_result_11 = get_stats_rb(rb_by_type_11['PLAYER&pos;=rb&week;"&gt;PLAYER'])

### Quarterbacks

In [ ]:
#Week 1
qb_table_1 = get_table(qb_html_1)
qb_by_type_1 = defaultdict(list)  
for tbl in qb_table_1:
    qb_by_type_1[table_type(tbl)].append(tbl)
qb_result_1 = get_stats_qb(qb_by_type_1['PLAYER&pos;=qb&week;"&gt;PLAYER'])

#Week 2
qb_table_2 = get_table(qb_html_2)   
qb_by_type_2 = defaultdict(list)  
for tbl in qb_table_2:
    qb_by_type_2[table_type(tbl)].append(tbl)
qb_result_2 = get_stats_qb(qb_by_type_2['PLAYER&pos;=qb&week;"&gt;PLAYER'])

#Week 3
qb_table_3 = get_table(qb_html_3)   
qb_by_type_3 = defaultdict(list)  
for tbl in qb_table_3:
    qb_by_type_3[table_type(tbl)].append(tbl)
qb_result_3 = get_stats_qb(qb_by_type_3['PLAYER&pos;=qb&week;"&gt;PLAYER'])

#Week 4
qb_table_4 = get_table(qb_html_4)   
qb_by_type_4 = defaultdict(list)  
for tbl in qb_table_4:
    qb_by_type_4[table_type(tbl)].append(tbl)
qb_result_4 = get_stats_qb(qb_by_type_4['PLAYER&pos;=qb&week;"&gt;PLAYER'])

#Week 5
qb_table_5 = get_table(qb_html_5)   
qb_by_type_5 = defaultdict(list)  
for tbl in qb_table_5:
    qb_by_type_5[table_type(tbl)].append(tbl)
qb_result_5 = get_stats_qb(qb_by_type_5['PLAYER&pos;=qb&week;"&gt;PLAYER'])

#Week 6
qb_table_6 = get_table(qb_html_6)   
qb_by_type_6 = defaultdict(list)  
for tbl in qb_table_6:
    qb_by_type_6[table_type(tbl)].append(tbl)
qb_result_6 = get_stats_qb(qb_by_type_6['PLAYER&pos;=qb&week;"&gt;PLAYER'])

#Week 7
qb_table_7 = get_table(qb_html_7)   
qb_by_type_7 = defaultdict(list)  
for tbl in qb_table_7:
    qb_by_type_7[table_type(tbl)].append(tbl)
qb_result_7 = get_stats_qb(qb_by_type_7['PLAYER&pos;=qb&week;"&gt;PLAYER'])

#Week 8
qb_table_8 = get_table(qb_html_8)   
qb_by_type_8 = defaultdict(list)  
for tbl in qb_table_8:
    qb_by_type_8[table_type(tbl)].append(tbl)
qb_result_8 = get_stats_qb(qb_by_type_8['PLAYER&pos;=qb&week;"&gt;PLAYER'])

#Week 9
qb_table_9 = get_table(qb_html_9)   
qb_by_type_9 = defaultdict(list)  
for tbl in qb_table_9:
    qb_by_type_9[table_type(tbl)].append(tbl)
qb_result_9 = get_stats_qb(qb_by_type_9['PLAYER&pos;=qb&week;"&gt;PLAYER'])

#Week 10
qb_table_10 = get_table(qb_html_10)   
qb_by_type_10 = defaultdict(list)  
for tbl in qb_table_10:
    qb_by_type_10[table_type(tbl)].append(tbl)
qb_result_10 = get_stats_qb(qb_by_type_10['PLAYER&pos;=qb&week;"&gt;PLAYER'])

#Week 11
qb_table_11 = get_table(qb_html_11)   
qb_by_type_11 = defaultdict(list)  
for tbl in qb_table_11:
    qb_by_type_11[table_type(tbl)].append(tbl)
qb_result_11 = get_stats_qb(qb_by_type_11['PLAYER&pos;=qb&week;"&gt;PLAYER'])

### Wide Recievers

In [ ]:
#Week 1
wr_table_1 = get_table(wr_html_1)
wr_by_type_1 = defaultdict(list)  
for tbl in wr_table_1:
    wr_by_type_1[table_type(tbl)].append(tbl)
wr_result_1 = get_stats_wr(wr_by_type_1['PLAYER&pos;=wr&week;"&gt;PLAYER'])

#Week 2
wr_table_2 = get_table(wr_html_2)   
wr_by_type_2 = defaultdict(list)  
for tbl in wr_table_2:
    wr_by_type_2[table_type(tbl)].append(tbl)
wr_result_2 = get_stats_wr(wr_by_type_2['PLAYER&pos;=wr&week;"&gt;PLAYER'])

#Week 3
wr_table_3 = get_table(wr_html_3)   
wr_by_type_3 = defaultdict(list)  
for tbl in wr_table_3:
    wr_by_type_3[table_type(tbl)].append(tbl)
wr_result_3 = get_stats_wr(wr_by_type_3['PLAYER&pos;=wr&week;"&gt;PLAYER'])

#Week 4
wr_table_4 = get_table(wr_html_4)   
wr_by_type_4 = defaultdict(list)  
for tbl in wr_table_4:
    wr_by_type_4[table_type(tbl)].append(tbl)
wr_result_4 = get_stats_wr(wr_by_type_4['PLAYER&pos;=wr&week;"&gt;PLAYER'])

#Week 5
wr_table_5 = get_table(wr_html_5)   
wr_by_type_5 = defaultdict(list)  
for tbl in wr_table_5:
    wr_by_type_5[table_type(tbl)].append(tbl)
wr_result_5 = get_stats_wr(wr_by_type_5['PLAYER&pos;=wr&week;"&gt;PLAYER'])

#Week 6
wr_table_6 = get_table(wr_html_6)   
wr_by_type_6 = defaultdict(list)  
for tbl in wr_table_6:
    wr_by_type_6[table_type(tbl)].append(tbl)
wr_result_6 = get_stats_wr(wr_by_type_6['PLAYER&pos;=wr&week;"&gt;PLAYER'])

#Week 7
wr_table_7 = get_table(wr_html_7)   
wr_by_type_7 = defaultdict(list)  
for tbl in wr_table_7:
    wr_by_type_7[table_type(tbl)].append(tbl)
wr_result_7 = get_stats_wr(wr_by_type_7['PLAYER&pos;=wr&week;"&gt;PLAYER'])

#Week 8
wr_table_8 = get_table(wr_html_8)   
wr_by_type_8 = defaultdict(list)  
for tbl in wr_table_8:
    wr_by_type_8[table_type(tbl)].append(tbl)
wr_result_8 = get_stats_wr(wr_by_type_8['PLAYER&pos;=wr&week;"&gt;PLAYER'])

#Week 9
wr_table_9 = get_table(wr_html_9)   
wr_by_type_9 = defaultdict(list)  
for tbl in wr_table_9:
    wr_by_type_9[table_type(tbl)].append(tbl)
wr_result_9 = get_stats_wr(wr_by_type_9['PLAYER&pos;=wr&week;"&gt;PLAYER'])

#Week 10
wr_table_10 = get_table(wr_html_10)   
wr_by_type_10 = defaultdict(list)  
for tbl in wr_table_10:
    wr_by_type_10[table_type(tbl)].append(tbl)
wr_result_10 = get_stats_wr(wr_by_type_10['PLAYER&pos;=wr&week;"&gt;PLAYER'])

#Week 11
wr_table_11 = get_table(wr_html_11)   
wr_by_type_11 = defaultdict(list)  
for tbl in wr_table_11:
    wr_by_type_11[table_type(tbl)].append(tbl)
wr_result_11 = get_stats_wr(wr_by_type_11['PLAYER&pos;=wr&week;"&gt;PLAYER'])

### Tight Ends

In [ ]:
#Week 1
te_table_1 = get_table(te_html_1)
te_by_type_1 = defaultdict(list)  
for tbl in te_table_1:
    te_by_type_1[table_type(tbl)].append(tbl)
te_result_1 = get_stats_te(te_by_type_1['PLAYER&pos;=te&week;"&gt;PLAYER'])

#Week 2
te_table_2 = get_table(te_html_2)   
te_by_type_2 = defaultdict(list)  
for tbl in te_table_2:
    te_by_type_2[table_type(tbl)].append(tbl)
te_result_2 = get_stats_te(te_by_type_2['PLAYER&pos;=te&week;"&gt;PLAYER'])

#Week 3
te_table_3 = get_table(te_html_3)   
te_by_type_3 = defaultdict(list)  
for tbl in te_table_3:
    te_by_type_3[table_type(tbl)].append(tbl)
te_result_3 = get_stats_te(te_by_type_3['PLAYER&pos;=te&week;"&gt;PLAYER'])

#Week 4
te_table_4 = get_table(te_html_4)   
te_by_type_4 = defaultdict(list)  
for tbl in te_table_4:
    te_by_type_4[table_type(tbl)].append(tbl)
te_result_4 = get_stats_te(te_by_type_4['PLAYER&pos;=te&week;"&gt;PLAYER'])

#Week 5
te_table_5 = get_table(te_html_5)   
te_by_type_5 = defaultdict(list)  
for tbl in te_table_5:
    te_by_type_5[table_type(tbl)].append(tbl)
te_result_5 = get_stats_te(te_by_type_5['PLAYER&pos;=te&week;"&gt;PLAYER'])

#Week 6
te_table_6 = get_table(te_html_6)   
te_by_type_6 = defaultdict(list)  
for tbl in te_table_6:
    te_by_type_6[table_type(tbl)].append(tbl)
te_result_6 = get_stats_te(te_by_type_6['PLAYER&pos;=te&week;"&gt;PLAYER'])

#Week 7
te_table_7 = get_table(te_html_7)   
te_by_type_7 = defaultdict(list)  
for tbl in te_table_7:
    te_by_type_7[table_type(tbl)].append(tbl)
te_result_7 = get_stats_te(te_by_type_7['PLAYER&pos;=te&week;"&gt;PLAYER'])

#Week 8
te_table_8 = get_table(te_html_8)   
te_by_type_8 = defaultdict(list)  
for tbl in te_table_8:
    te_by_type_8[table_type(tbl)].append(tbl)
te_result_8 = get_stats_te(te_by_type_8['PLAYER&pos;=te&week;"&gt;PLAYER'])

#Week 9
te_table_9 = get_table(te_html_9)   
te_by_type_9 = defaultdict(list)  
for tbl in te_table_9:
    te_by_type_9[table_type(tbl)].append(tbl)
te_result_9 = get_stats_te(te_by_type_9['PLAYER&pos;=te&week;"&gt;PLAYER'])

#Week 10
te_table_10 = get_table(te_html_10)   
te_by_type_10 = defaultdict(list)  
for tbl in te_table_10:
    te_by_type_10[table_type(tbl)].append(tbl)
te_result_10 = get_stats_te(te_by_type_10['PLAYER&pos;=te&week;"&gt;PLAYER'])

#Week 11
te_table_11 = get_table(te_html_11)   
te_by_type_11 = defaultdict(list)  
for tbl in te_table_11:
    te_by_type_11[table_type(tbl)].append(tbl)
te_result_11 = get_stats_te(te_by_type_11['PLAYER&pos;=te&week;"&gt;PLAYER'])

### Defense

In [ ]:
#Week 1
df_table_1 = get_table(df_html_1)
df_by_type_1 = defaultdict(list)  
for tbl in df_table_1:
    df_by_type_1[table_type(tbl)].append(tbl)
df_result_1 = get_stats_df(df_by_type_1['PLAYER&pos;=df&week;"&gt;PLAYER'])

#Week 2
df_table_2 = get_table(df_html_2)   
df_by_type_2 = defaultdict(list)  
for tbl in df_table_2:
    df_by_type_2[table_type(tbl)].append(tbl)
df_result_2 = get_stats_df(df_by_type_2['PLAYER&pos;=df&week;"&gt;PLAYER'])

#Week 3
df_table_3 = get_table(df_html_3)   
df_by_type_3 = defaultdict(list)  
for tbl in df_table_3:
    df_by_type_3[table_type(tbl)].append(tbl)
df_result_3 = get_stats_df(df_by_type_3['PLAYER&pos;=df&week;"&gt;PLAYER'])

#Week 4
df_table_4 = get_table(df_html_4)   
df_by_type_4 = defaultdict(list)  
for tbl in df_table_4:
    df_by_type_4[table_type(tbl)].append(tbl)
df_result_4 = get_stats_df(df_by_type_4['PLAYER&pos;=df&week;"&gt;PLAYER'])

#Week 5
df_table_5 = get_table(df_html_5)   
df_by_type_5 = defaultdict(list)  
for tbl in df_table_5:
    df_by_type_5[table_type(tbl)].append(tbl)
df_result_5 = get_stats_df(df_by_type_5['PLAYER&pos;=df&week;"&gt;PLAYER'])

#Week 6
df_table_6 = get_table(df_html_6)   
df_by_type_6 = defaultdict(list)  
for tbl in df_table_6:
    df_by_type_6[table_type(tbl)].append(tbl)
df_result_6 = get_stats_df(df_by_type_6['PLAYER&pos;=df&week;"&gt;PLAYER'])

#Week 7
df_table_7 = get_table(df_html_7)   
df_by_type_7 = defaultdict(list)  
for tbl in df_table_7:
    df_by_type_7[table_type(tbl)].append(tbl)
df_result_7 = get_stats_df(df_by_type_7['PLAYER&pos;=df&week;"&gt;PLAYER'])

#Week 8
df_table_8 = get_table(df_html_8)   
df_by_type_8 = defaultdict(list)  
for tbl in df_table_8:
    df_by_type_8[table_type(tbl)].append(tbl)
df_result_8 = get_stats_df(df_by_type_8['PLAYER&pos;=df&week;"&gt;PLAYER'])

#Week 9
df_table_9 = get_table(df_html_9)   
df_by_type_9 = defaultdict(list)  
for tbl in df_table_9:
    df_by_type_9[table_type(tbl)].append(tbl)
df_result_9 = get_stats_df(df_by_type_9['PLAYER&pos;=df&week;"&gt;PLAYER'])

#Week 10
df_table_10 = get_table(df_html_10)   
df_by_type_10 = defaultdict(list)  
for tbl in df_table_10:
    df_by_type_10[table_type(tbl)].append(tbl)
df_result_10 = get_stats_df(df_by_type_10['PLAYER&pos;=df&week;"&gt;PLAYER'])

#Week 11
df_table_11 = get_table(df_html_11)   
df_by_type_11 = defaultdict(list)  
for tbl in df_table_11:
    df_by_type_11[table_type(tbl)].append(tbl)
df_result_11 = get_stats_df(df_by_type_11['PLAYER&pos;=df&week;"&gt;PLAYER'])

## Create DataFrames:

### Running Backs:

In [ ]:
rb_week1=make_df(rb_result_1)
rb_week2=make_df(rb_result_2)
rb_week3=make_df(rb_result_3)
rb_week4=make_df(rb_result_4)
rb_week5=make_df(rb_result_5)
rb_week6=make_df(rb_result_6)
rb_week7=make_df(rb_result_7)
rb_week8=make_df(rb_result_8)
rb_week9=make_df(rb_result_9)
rb_week10=make_df(rb_result_10)
rb_week11=make_df(rb_result_11)

rb_mid1=pd.DataFrame(rb_week1.FPTS)
rb_mid2=pd.DataFrame(rb_week2.FPTS)
rb_mid3=pd.DataFrame(rb_week3.FPTS)
rb_mid4=pd.DataFrame(rb_week4.FPTS)
rb_mid5=pd.DataFrame(rb_week5.FPTS)
rb_mid6=pd.DataFrame(rb_week6.FPTS)
rb_mid7=pd.DataFrame(rb_week7.FPTS)
rb_mid8=pd.DataFrame(rb_week8.FPTS)
rb_mid9=pd.DataFrame(rb_week9.FPTS)
rb_mid10=pd.DataFrame(rb_week10.FPTS)
rb_mid11=pd.DataFrame(rb_week11.FPTS)
rb_final=pd.concat([rb_mid1,rb_mid2,rb_mid3,rb_mid4,rb_mid5,rb_mid6,rb_mid7,rb_mid8,rb_mid9,rb_mid10,rb_mid11],axis=1)
rb_final.columns=[1,2,3,4,5,6,7,8,9,10,11]
rb_final

### Quarterbacks:

In [ ]:
qb_week1=make_df(qb_result_1)
qb_week2=make_df(qb_result_2)
qb_week3=make_df(qb_result_3)
qb_week4=make_df(qb_result_4)
qb_week5=make_df(qb_result_5)
qb_week6=make_df(qb_result_6)
qb_week7=make_df(qb_result_7)
qb_week8=make_df(qb_result_8)
qb_week9=make_df(qb_result_9)
qb_week10=make_df(qb_result_10)
qb_week11=make_df(qb_result_11)

qb_mid1=pd.DataFrame(qb_week1.FPTS)
qb_mid2=pd.DataFrame(qb_week2.FPTS)
qb_mid3=pd.DataFrame(qb_week3.FPTS)
qb_mid4=pd.DataFrame(qb_week4.FPTS)
qb_mid5=pd.DataFrame(qb_week5.FPTS)
qb_mid6=pd.DataFrame(qb_week6.FPTS)
qb_mid7=pd.DataFrame(qb_week7.FPTS)
qb_mid8=pd.DataFrame(qb_week8.FPTS)
qb_mid9=pd.DataFrame(qb_week9.FPTS)
qb_mid10=pd.DataFrame(qb_week10.FPTS)
qb_mid11=pd.DataFrame(qb_week11.FPTS)
qb_final=pd.concat([qb_mid1,qb_mid2,qb_mid3,qb_mid4,qb_mid5,qb_mid6,qb_mid7,qb_mid8,qb_mid9,qb_mid10,qb_mid11],axis=1)
qb_final.columns=[1,2,3,4,5,6,7,8,9,10,11]
qb_final

### Wide Recievers:

In [ ]:
wr_week1=make_df(wr_result_1)
wr_week2=make_df(wr_result_2)
wr_week3=make_df(wr_result_3)
wr_week4=make_df(wr_result_4)
wr_week5=make_df(wr_result_5)
wr_week6=make_df(wr_result_6)
wr_week7=make_df(wr_result_7)
wr_week8=make_df(wr_result_8)
wr_week9=make_df(wr_result_9)
wr_week10=make_df(wr_result_10)
wr_week11=make_df(wr_result_11)

wr_mid1=pd.DataFrame(wr_week1.FPTS)
wr_mid2=pd.DataFrame(wr_week2.FPTS)
wr_mid3=pd.DataFrame(wr_week3.FPTS)
wr_mid4=pd.DataFrame(wr_week4.FPTS)
wr_mid5=pd.DataFrame(wr_week5.FPTS)
wr_mid6=pd.DataFrame(wr_week6.FPTS)
wr_mid7=pd.DataFrame(wr_week7.FPTS)
wr_mid8=pd.DataFrame(wr_week8.FPTS)
wr_mid9=pd.DataFrame(wr_week9.FPTS)
wr_mid10=pd.DataFrame(wr_week10.FPTS)
wr_mid11=pd.DataFrame(wr_week11.FPTS)
wr_final=pd.concat([wr_mid1,wr_mid2,wr_mid3,wr_mid4,wr_mid5,wr_mid6,wr_mid7,wr_mid8,wr_mid9,wr_mid10,wr_mid11],axis=1)
wr_final.columns=[1,2,3,4,5,6,7,8,9,10,11]
wr_final

### Tight Ends

In [ ]:
te_week1=make_df(te_result_1)
te_week2=make_df(te_result_2)
te_week3=make_df(te_result_3)
te_week4=make_df(te_result_4)
te_week5=make_df(te_result_5)
te_week6=make_df(te_result_6)
te_week7=make_df(te_result_7)
te_week8=make_df(te_result_8)
te_week9=make_df(te_result_9)
te_week10=make_df(te_result_10)
te_week11=make_df(te_result_11)

te_mid1=pd.DataFrame(te_week1.FPTS)
te_mid2=pd.DataFrame(te_week2.FPTS)
te_mid3=pd.DataFrame(te_week3.FPTS)
te_mid4=pd.DataFrame(te_week4.FPTS)
te_mid5=pd.DataFrame(te_week5.FPTS)
te_mid6=pd.DataFrame(te_week6.FPTS)
te_mid7=pd.DataFrame(te_week7.FPTS)
te_mid8=pd.DataFrame(te_week8.FPTS)
te_mid9=pd.DataFrame(te_week9.FPTS)
te_mid10=pd.DataFrame(te_week10.FPTS)
te_mid11=pd.DataFrame(te_week11.FPTS)
te_final=pd.concat([te_mid1,te_mid2,te_mid3,te_mid4,te_mid5,te_mid6,te_mid7,te_mid8,te_mid9,te_mid10,te_mid11],axis=1)
te_final.columns=[1,2,3,4,5,6,7,8,9,10,11]
te_final

### Defence

In [ ]:
df_week1=make_dfdf(df_result_1)
df_week2=make_dfdf(df_result_2)
df_week3=make_dfdf(df_result_3)
df_week4=make_dfdf(df_result_4)
df_week5=make_dfdf(df_result_5)
df_week6=make_dfdf(df_result_6)
df_week7=make_dfdf(df_result_7)
df_week8=make_dfdf(df_result_8)
df_week9=make_dfdf(df_result_9)
df_week10=make_dfdf(df_result_10)
df_week11=make_dfdf(df_result_11)

df_mid1=pd.DataFrame(df_week1.FPTS)
df_mid2=pd.DataFrame(df_week2.FPTS)
df_mid3=pd.DataFrame(df_week3.FPTS)
df_mid4=pd.DataFrame(df_week4.FPTS)
df_mid5=pd.DataFrame(df_week5.FPTS)
df_mid6=pd.DataFrame(df_week6.FPTS)
df_mid7=pd.DataFrame(df_week7.FPTS)
df_mid8=pd.DataFrame(df_week8.FPTS)
df_mid9=pd.DataFrame(df_week9.FPTS)
df_mid10=pd.DataFrame(df_week10.FPTS)
df_mid11=pd.DataFrame(df_week11.FPTS)
df_final=pd.concat([df_mid1,df_mid2,df_mid3,df_mid4,df_mid5,df_mid6,df_mid7,df_mid8,df_mid9,df_mid10,df_mid11],axis=1)
df_final.columns=[1,2,3,4,5,6,7,8,9,10,11]
df_final

## Player Pricing Data

### Import Pricing Data

In [ ]:
f = open('DKSalaries2.csv')
csv_f = csv.reader(f)
prices = []
for row in csv_f:
  prices.append(row)
pricedf=pd.DataFrame(prices)
pricedf=pricedf.drop(0)
pricedf=pricedf.drop(3,1)
pricedf=pricedf.drop(4,1)
pricedf=pricedf.drop(5,1)
pricedf.columns=['POSITION','PLAYER','SALARY']
pricedf=pricedf.sort_values(by=['POSITION','PLAYER'])
pricedf=pricedf.reset_index(drop=True)
index=pricedf['PLAYER']
pricedf.index = index
pricedf=pricedf.drop('PLAYER',1)

### Separate Pricing Data By Player Type

In [ ]:
DSTPrice=pricedf[pricedf['POSITION']=='DST']
QBPrice=pricedf[pricedf['POSITION']=='QB']
RBPrice=pricedf[pricedf['POSITION']=='RB']
WRPrice=pricedf[pricedf['POSITION']=='WR']
TEPrice=pricedf[pricedf['POSITION']=='TE']

# Analysis:

## Columnwise Means, Sums, Variences:

### Running Backs

In [ ]:
rb_col_sum=rb_final.sum(skipna=True)
rb_col_sum.name='SUM'
rb_col_sum1=pd.DataFrame(rb_col_sum).T
rb_col_mean=rb_final.mean(skipna=True)
rb_col_mean.name='MEAN'
rb_col_mean1=pd.DataFrame(rb_col_mean).T
rb_col_sd=rb_final.std(skipna=True)
rb_col_sd.name='SD'
rb_col_sd1=pd.DataFrame(rb_col_sd).T
rb_col_stats=rb_col_sum1.append([rb_col_mean1,rb_col_sd1])
rb_col_stats

### Quarterbacks

In [ ]:
qb_col_sum=qb_final.sum(skipna=True)
qb_col_sum.name='SUM'
qb_col_sum1=pd.DataFrame(qb_col_sum).T
qb_col_mean=qb_final.mean(skipna=True)
qb_col_mean.name='MEAN'
qb_col_mean1=pd.DataFrame(qb_col_mean).T
qb_col_sd=qb_final.std(skipna=True)
qb_col_sd.name='SD'
qb_col_sd1=pd.DataFrame(qb_col_sd).T
qb_col_stats=qb_col_sum1.append([qb_col_mean1,qb_col_sd1])
qb_col_stats

### Wide Recievers

In [ ]:
wr_col_sum=wr_final.sum(skipna=True)
wr_col_sum.name='SUM'
wr_col_sum1=pd.DataFrame(wr_col_sum).T
wr_col_mean=wr_final.mean(skipna=True)
wr_col_mean.name='MEAN'
wr_col_mean1=pd.DataFrame(wr_col_mean).T
wr_col_sd=wr_final.std(skipna=True)
wr_col_sd.name='SD'
wr_col_sd1=pd.DataFrame(wr_col_sd).T
wr_col_stats=wr_col_sum1.append([wr_col_mean1,wr_col_sd1])
wr_col_stats

### Tight Ends

In [ ]:
te_col_sum=te_final.sum(skipna=True)
te_col_sum.name='SUM'
te_col_sum1=pd.DataFrame(te_col_sum).T
te_col_mean=te_final.mean(skipna=True)
te_col_mean.name='MEAN'
te_col_mean1=pd.DataFrame(te_col_mean).T
te_col_sd=te_final.std(skipna=True)
te_col_sd.name='SD'
te_col_sd1=pd.DataFrame(te_col_sd).T
te_col_stats=te_col_sum1.append([te_col_mean1,te_col_sd1])
te_col_stats

### Defence

In [ ]:
df_col_sum=df_final.sum(skipna=True)
df_col_sum.name='SUM'
df_col_sum1=pd.DataFrame(df_col_sum).T
df_col_mean=df_final.mean(skipna=True)
df_col_mean.name='MEAN'
df_col_mean1=pd.DataFrame(df_col_mean).T
df_col_sd=df_final.std(skipna=True)
df_col_sd.name='SD'
df_col_sd1=pd.DataFrame(df_col_sd).T
df_col_stats=df_col_sum1.append([df_col_mean1,df_col_sd1])
df_col_stats

## Rowwise Means, Sums, Variences:

### Running Backs

In [ ]:
rb_row_sum=rb_final.sum(axis=1,skipna=True)
rb_row_sum.name='SUM'
rb_row_sum1=pd.DataFrame(rb_row_sum)
rb_row_mean=rb_final.mean(axis=1,skipna=True)
rb_row_mean.name='MEAN'
rb_row_mean1=pd.DataFrame(rb_row_mean)
rb_row_sd=rb_final.std(axis=1,skipna=True)
rb_row_sd.name='SD'
rb_row_sd1=pd.DataFrame(rb_row_sd)
rb_row_max=rb_final.max(axis=1,skipna=True)
rb_row_max.name='MAX'
rb_row_max1=pd.DataFrame(rb_row_max)
rb_row_stats=pd.concat([rb_row_max1,rb_row_sum1,rb_row_mean1,rb_row_sd1],axis=1)

#Cleanup
rb_row_stats1=rb_row_stats.sort_values(by='MEAN', ascending=False)
rb_row_stats2=rb_row_stats1[:60]
rb_top_ten=rb_row_stats1[:10]
rb_row_stats3=rb_row_stats.sort_values(by='MAX', ascending=False)
rb_top_ten2=rb_row_stats3[:10]
rb_row_stats2

#### Merge With Pricing Data

In [ ]:
rb_final2=pd.concat([rb_row_stats2,RBPrice],axis=1)
rb_final2['SALARY']=rb_final2['SALARY'].astype(float)
Avg1=rb_final2['MEAN']/rb_final2['SALARY']
Avg=pd.DataFrame(Avg1)
Avg.columns=['AVG_PTs/$']
rb_final2=pd.concat([rb_final2,Avg],axis=1)
rb_final3=rb_final2.dropna()
rb_final4=rb_final3[rb_final3['SUM']>=100]
rb_final5=rb_final4.sort_values(by='AVG_PTs/$', ascending=False)
rb_top_ten_pricing=rb_final5[:10]
rb_final5

### Quarterbacks

In [ ]:
qb_row_sum=qb_final.sum(axis=1,skipna=True)
qb_row_sum.name='SUM'
qb_row_sum1=pd.DataFrame(qb_row_sum)
qb_row_mean=qb_final.mean(axis=1,skipna=True)
qb_row_mean.name='MEAN'
qb_row_mean1=pd.DataFrame(qb_row_mean)
qb_row_sd=qb_final.std(axis=1,skipna=True)
qb_row_sd.name='SD'
qb_row_sd1=pd.DataFrame(qb_row_sd)
qb_row_max=qb_final.max(axis=1,skipna=True)
qb_row_max.name='MAX'
qb_row_max1=pd.DataFrame(qb_row_max)
qb_row_stats=pd.concat([qb_row_max1,qb_row_sum1,qb_row_mean1,qb_row_sd1],axis=1)

#Cleanup
qb_row_stats1=qb_row_stats.sort_values(by='MEAN', ascending=False)
qb_row_stats2=qb_row_stats1[:30]
qb_top_ten=qb_row_stats1[:10]
qb_row_stats3=qb_row_stats.sort_values(by='MAX', ascending=False)
qb_top_ten2=qb_row_stats3[:10]
qb_row_stats2

#### Merge With Pricing Data

In [ ]:
qb_final2=pd.concat([qb_row_stats2,QBPrice],axis=1)
qb_final2['SALARY']=qb_final2['SALARY'].astype(float)
Avg1=qb_final2['MEAN']/qb_final2['SALARY']
Avg=pd.DataFrame(Avg1)
Avg.columns=['AVG_PTs/$']
qb_final2=pd.concat([qb_final2,Avg],axis=1)
qb_final3=qb_final2.dropna()
qb_final4=qb_final3[qb_final3['SUM']>=150]
qb_final5=qb_final4.sort_values(by='AVG_PTs/$', ascending=False)
qb_top_ten_pricing=qb_final5[:10]
qb_final5

### Wide Recievers

In [ ]:
wr_row_sum=wr_final.sum(axis=1,skipna=True)
wr_row_sum.name='SUM'
wr_row_sum1=pd.DataFrame(wr_row_sum)
wr_row_mean=wr_final.mean(axis=1,skipna=True)
wr_row_mean.name='MEAN'
wr_row_mean1=pd.DataFrame(wr_row_mean)
wr_row_sd=wr_final.std(axis=1,skipna=True)
wr_row_sd.name='SD'
wr_row_sd1=pd.DataFrame(wr_row_sd)
wr_row_max=wr_final.max(axis=1,skipna=True)
wr_row_max.name='MAX'
wr_row_max1=pd.DataFrame(wr_row_max)
wr_row_stats=pd.concat([wr_row_max1,wr_row_sum1,wr_row_mean1,wr_row_sd1],axis=1)

#Cleanup
wr_row_stats1=wr_row_stats.sort_values(by='MEAN', ascending=False)
wr_row_stats2=wr_row_stats1[:60]
wr_top_ten=wr_row_stats1[:10]
wr_row_stats3=wr_row_stats.sort_values(by='MAX', ascending=False)
wr_top_ten2=wr_row_stats3[:10]
wr_row_stats2

#### Merge With Pricing Data

In [ ]:
wr_final2=pd.concat([wr_row_stats2,WRPrice],axis=1)
wr_final2['SALARY']=wr_final2['SALARY'].astype(float)
Avg1=wr_final2['MEAN']/wr_final2['SALARY']
Avg=pd.DataFrame(Avg1)
Avg.columns=['AVG_PTs/$']
wr_final2=pd.concat([wr_final2,Avg],axis=1)
wr_final3=wr_final2.dropna()
wr_final4=wr_final3[wr_final3['SUM']>=100]
wr_final5=wr_final4.sort_values(by='AVG_PTs/$', ascending=False)
wr_top_ten_pricing=wr_final5[:10]
wr_final5

### Tight Ends

In [ ]:
te_row_sum=te_final.sum(axis=1,skipna=True)
te_row_sum.name='SUM'
te_row_sum1=pd.DataFrame(te_row_sum)
te_row_mean=te_final.mean(axis=1,skipna=True)
te_row_mean.name='MEAN'
te_row_mean1=pd.DataFrame(te_row_mean)
te_row_sd=te_final.std(axis=1,skipna=True)
te_row_sd.name='SD'
te_row_sd1=pd.DataFrame(te_row_sd)
te_row_max=te_final.max(axis=1,skipna=True)
te_row_max.name='MAX'
te_row_max1=pd.DataFrame(te_row_max)
te_row_stats=pd.concat([te_row_max1,te_row_sum1,te_row_mean1,te_row_sd1],axis=1)

#Cleanup
te_row_stats1=te_row_stats.sort_values(by='MEAN', ascending=False)
te_row_stats2=te_row_stats1[:30]
te_top_ten=te_row_stats1[:10]
te_row_stats3=te_row_stats.sort_values(by='MAX', ascending=False)
te_top_ten2=te_row_stats3[:10]
te_row_stats2

#### Merge With Pricing Data

In [ ]:
te_final2=pd.concat([te_row_stats2,TEPrice],axis=1)
te_final2['SALARY']=te_final2['SALARY'].astype(float)
Avg1=te_final2['MEAN']/te_final2['SALARY']
Avg=pd.DataFrame(Avg1)
Avg.columns=['AVG_PTs/$']
te_final2=pd.concat([te_final2,Avg],axis=1)
te_final3=te_final2.dropna()
te_final4=te_final3[te_final3['SUM']>=75]
te_final5=te_final4.sort_values(by='AVG_PTs/$', ascending=False)
te_top_ten_pricing=te_final5[:10]
te_final5

### Defence

In [ ]:
df_row_sum=df_final.sum(axis=1,skipna=True)
df_row_sum.name='SUM'
df_row_sum1=pd.DataFrame(df_row_sum)
df_row_mean=df_final.mean(axis=1,skipna=True)
df_row_mean.name='MEAN'
df_row_mean1=pd.DataFrame(df_row_mean)
df_row_sd=df_final.std(axis=1,skipna=True)
df_row_sd.name='SD'
df_row_sd1=pd.DataFrame(df_row_sd)
df_row_max=df_final.max(axis=1,skipna=True)
df_row_max.name='MAX'
df_row_max1=pd.DataFrame(df_row_max)
df_row_stats=pd.concat([df_row_max1,df_row_sum1,df_row_mean1,df_row_sd1],axis=1)

#Cleanup
df_row_stats1=df_row_stats.sort_values(by='MEAN', ascending=False)
df_row_stats2=df_row_stats1[:32]
df_top_ten=df_row_stats1[:10]
df_row_stats3=df_row_stats.sort_values(by='MAX', ascending=False)
df_top_ten2=df_row_stats3[:10]
df_row_stats2

#### Merge With Pricing Data

In [ ]:
df_final2=pd.concat([df_row_stats,DSTPrice],axis=1)
df_final2['SALARY']=df_final2['SALARY'].astype(float)
Avg1=df_final2['MEAN']/df_final2['SALARY']
Avg=pd.DataFrame(Avg1)
Avg.columns=['AVG_PTs/$']
df_final3=pd.concat([df_final2,Avg],axis=1)
df_final5=df_final3.sort_values(by='AVG_PTs/$', ascending=False)
df_top_ten_pricing=df_final5[:10]
df_final5

## Master List

In [ ]:
masterlist=pd.concat([qb_final5,rb_final5,wr_final5,te_final4,df_final5],axis=0)
masterlist2=masterlist.sort_values(by='AVG', ascending=False)
masterlist2
masterlist2.to_csv('master.csv')

## Plots

### Points per Week by Player Type

In [ ]:
weeks=[1,2,3,4,5,6,7,8,9,10,11]
fig = plt.figure(figsize=(7,5))
plt.xlabel('Week')
plt.ylabel('Average Points')
plt.title('Average Points Per Week by Player Type')
plt.plot(weeks, rb_col_stats.loc['MEAN',:],'r')
plt.plot(weeks, qb_col_stats.loc['MEAN',:],'b')
plt.plot(weeks, wr_col_stats.loc['MEAN',:],'g')
plt.plot(weeks, te_col_stats.loc['MEAN',:],'m')
plt.plot(weeks, df_col_stats.loc['MEAN',:],'k')
plt.legend(['RB', 'QB','WR','TE','DF'], loc=1,bbox_to_anchor=(1.23, .385))
plt.show()

### Bar Plot of Top Ten Players

In [ ]:
fig, ax = plt.subplots()
x=np.arange(10)
ax.set_ylabel('Average Points')
plt.title('Top Ten Players by Avg Points')
w=0.2
ax.set(xticks=x+w, xticklabels=range(1,11), xlim=[0, 11])
plt.bar(x-2*w, rb_top_ten['MEAN'], width=w, color='r')
plt.bar(x-w, qb_top_ten['MEAN'], width=w, color='b')
plt.bar(x, wr_top_ten['MEAN'], width=w, color='g')
plt.bar(x+w, te_top_ten['MEAN'], width=w, color='m')
plt.bar(x+2*w, df_top_ten['MEAN'], width=w, color='orange')
ax.autoscale(tight=True)
plt.legend(['RB', 'QB','WR','TE','DF'], loc=1,bbox_to_anchor=(1.23, .385))
plt.show()

In [ ]:
fig, ax = plt.subplots()
x=np.arange(10)
ax.set_ylabel('Average Points')
plt.title('Top Ten Players by Avg Points per Dollar')
w=0.2
ax.set(xticks=x+w, xticklabels=range(1,11), xlim=[0, 11])
plt.bar(x-2*w, rb_top_ten_pricing['AVG_PTs/$'], width=w, color='r')
plt.bar(x-w, qb_top_ten_pricing['AVG_PTs/$'], width=w, color='b')
plt.bar(x, wr_top_ten_pricing['AVG_PTs/$'], width=w, color='g')
plt.bar(x+w, te_top_ten_pricing['AVG_PTs/$'], width=w, color='m')
plt.bar(x+2*w, df_top_ten_pricing['AVG_PTs/$'], width=w, color='orange')
ax.autoscale(tight=True)
plt.legend(['RB', 'QB','WR','TE','DF'], loc=1,bbox_to_anchor=(1.23, .385))
plt.show()

### Scatter Plot of Avg Points vs Salary

In [ ]:
fig = plt.figure(figsize=(7,5))
plt.scatter(rb_final5['MEAN'], rb_final5['SALARY'], color='r')
plt.scatter(qb_final5['MEAN'], qb_final5['SALARY'], color='b')
plt.scatter(wr_final5['MEAN'], wr_final5['SALARY'], color='g')
plt.scatter(te_final5['MEAN'], te_final5['SALARY'], color='m')
plt.scatter(df_final5['MEAN'], df_final5['SALARY'], color='orange')
plt.xlabel('Mean')
plt.ylabel('Salary')
plt.title('Avg Points vs Salary')
plt.legend(['RB', 'QB','WR','TE','DF'], loc=1,bbox_to_anchor=(1.23, .475))
plt.show()

### Scoring:

In [ ]:
"""Running Back"""
x = rb_row_stats2['MEAN']

plt.hist(x, 20, range=(0, 30), histtype='stepfilled',
         align='mid', color='r', label='Mean Points')

plt.legend()
plt.title('Running Back Scoring')
plt.show()

"""Quarteback"""
x = qb_row_stats2['MEAN']

plt.hist(x, 20, range=(0, 40), histtype='stepfilled',
         align='mid', color='b', label='Mean Points')

plt.legend()
plt.title('Quarterback Scoring')
plt.show()

"""Wide Reciever"""
x = wr_row_stats2['MEAN']

plt.hist(x, 20, range=(0, 30), histtype='stepfilled',
         align='mid', color='g', label='Mean Points')

plt.legend()
plt.title('Wide Reciever Scoring')
plt.show()

"""Tight End"""
x = te_row_stats2['MEAN']

plt.hist(x, 20, range=(0, 30), histtype='stepfilled',
         align='mid', color='m', label='Mean Points')

plt.legend()
plt.title('Tight End Scoring')
plt.show()

"""Defence"""
x = df_row_stats2['MEAN']

plt.hist(x, 20, range=(0, 15), histtype='stepfilled',
         align='mid', color='orange', label='Mean Points')

plt.legend()
plt.title('Defense Scoring')
plt.show()

# Results:

Choose team based on highest Avg Points/Dollar Spent
DraftKings Budget=$50000
Remove players who are injured or not expected to recieve regular playing time
Until Budget is met, players may be replaced by players with lower AVG/Dollar, but higher MEAN scores

## The Team:

In [ ]:
Players=['Tom Brady','DeMarco Murray','Lamar Miller','Larry Fitzgerald','Stefon Diggs','John Brown','Gary Barnidge','Danny Woodhead','DEN/Denver']
POSITION=['QB','RB','RB','WR','WR','WR','TE','RB','DF']
MEAN=[24.3,16.66666667,16.7,20.7,15.57142857,14.11111111,15.7,15.2,11.33333333]
PRICE=[8000,5800,5800,7400,4700,4800,5300,4800,3400]
POINTS=[23,3,4,17,7,15,16,11,5]
Team1=pd.DataFrame([Players,POSITION,MEAN,PRICE,POINTS])
Team=Team1.T
Team.index=Players
Team=Team.drop(0,1)
Team.columns=['POSITION','MEAN','PRICE','SCORE']
Diff=Team.SCORE-Team.MEAN
Team2=pd.concat([Team,Diff], axis=1)
Team2.columns=['POSITION','MEAN','PRICE','W12 SCORE','DIFFERENCE']
Team2

### Predicted vs Actual Performance

In [ ]:
n_groups = 9
fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.35

predicted = plt.bar(index, MEAN, bar_width, color='b',label='Predicted')

actual = plt.bar(index + bar_width, POINTS, bar_width, color='r', label='Actual')

plt.xlabel('Player')
plt.ylabel('Scores')
plt.title('Predicted vs. Actual Performance')
plt.xticks(index + bar_width, (Players), rotation=65)
plt.legend()

plt.tight_layout()
plt.show()

### Player Contribution to Point Total

In [ ]:
colors = ['blue', 'pink', 'orange', 'red', 'cyan', 'green', 'yellow', 'purple', 'lawngreen']

plt.pie(Team2['W12 SCORE'], colors=colors, labels=Team2.index, autopct='%.1f%%')
plt.title('Point Total')
plt.show()

In [ ]:
colors = ['blue', 'pink', 'orange', 'red', 'cyan', 'green', 'yellow', 'purple', 'lawngreen']

plt.pie(Team2['MEAN'], colors=colors, labels=Team2.index, autopct='%.1f%%')
plt.title('Expected Point Total')
plt.show()

## Picking a Better Team:

Given the underwhelming performance of a couple players, we returned to the data and noticed that those players had high variances...
We adjusted our team to take variance into consideration and got:

## The New Team:

In [ ]:
NewPlayers=['Tom Brady','Mark Ingram','LeSean McCoy','Brandon Marshall','Travis Benjamin','Stefon Diggs','Travis Kelce','Latavius Murray','DEN/Denver']
NewPOSITION=['QB','RB','RB','WR','WR','WR','TE','RB','DF']
NewMEAN=[24.3,17.1,16.375,18.4,14.5,15.57142857,13.1,13.6,11.33333333]
SD=[5.375872022,4.408325457,4.983902659,5.440588203,6.603870919,6.704653679,6.384877968,4.273952113,5.385164807]
NewPRICE=[8000,6400,5700,7100,5200,4700,4700,4800,3400]
NewPOINTS=[23,16,19,34,22,7,17,9,5]
NewTeam1=pd.DataFrame([NewPlayers,NewPOSITION,NewMEAN,SD,NewPRICE,NewPOINTS])
NewTeam=NewTeam1.T
NewTeam.index=NewPlayers
NewTeam=NewTeam.drop(0,1)
NewTeam.columns=['POSITION','MEAN','SD','PRICE','SCORE']
Diff=NewTeam.SCORE-NewTeam.MEAN
NewTeam2=pd.concat([NewTeam,Diff], axis=1)
NewTeam2.columns=['POSITION','MEAN','SD','PRICE','W12 SCORE','DIFFERENCE']
NewTeam2


### Predicted vs Actual Performance

In [ ]:
n_groups = 9
fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.35

predicted = plt.bar(index, NewMEAN, bar_width, color='b',label='Predicted')
actual = plt.bar(index + bar_width, NewPOINTS, bar_width, color='r', label='Actual')

plt.xlabel('Player')
plt.ylabel('Scores')
plt.title('Predicted vs. Actual Performance')
plt.xticks(index + bar_width, (NewPlayers), rotation=65)
plt.legend()
plt.tight_layout()
plt.show()

### Player Contribution to Point Total

In [ ]:
colors = ['blue', 'pink', 'orange', 'red', 'cyan', 'green', 'yellow', 'purple', 'lawngreen']

plt.pie(NewTeam2['W12 SCORE'], colors=colors, labels=NewTeam2.index, autopct='%.1f%%')
plt.title('Point Total')
plt.show()

In [ ]:
colors = ['blue', 'pink', 'orange', 'red', 'cyan', 'green', 'yellow', 'purple', 'lawngreen']

plt.pie(NewTeam2['MEAN'], colors=colors, labels=NewTeam2.index, autopct='%.1f%%')
plt.title('Expected Point Total')
plt.show()